# CS3244 Machine Learning Team Project
## Prediction of the number of dengue cases in Singapore

First, we shall import necessary Python libraries.

In [2]:
# Import data processing tools
import pandas as pd
import numpy as np

import Levenshtein as lev

import csv, json, sys
from fuzzywuzzy import fuzz

import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPoint
from shapely.ops import nearest_points

# Import visualization tools
import matplotlib.pyplot as plt

# Import utility tools
import requests
from math import *
from datetime import datetime
from datetime import timedelta

# Import ML tools
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


We shall read from various CSV files to extract dataframe for each feature in the following cells:
* Weekly **dengue** cases (*dengue_df*)
* Annual **population** (*population_df*)
* Monthly mean **sunshine** hours (*sunshine_df*)
* Monthly **surface air temperature** (*surface_air_temperature_df*)

In [162]:
# Read weekly dengue cases from csv files
raw_dengue_df = pd.read_csv(
    "Datasets/Dengue Cluster Data/weekly-number-of-dengue-and-dengue-haemorrhagic-fever-cases.csv")

# Clean data by filtering by type_dengue and remove the redundant column
dengue_df = raw_dengue_df[raw_dengue_df['type_dengue'].map(
    lambda x: str(x) == "Dengue")].drop('type_dengue', 1).reset_index()

# Rename eweek to week
dengue_df = dengue_df.rename(columns={"eweek": "week"}, errors="raise")

weekly_2019_df = pd.read_csv("Datasets/Dengue Cluster Data/2019_weekly_dengue.csv")
dengue_final_df = dengue_df.append([weekly_2019_df])

# Display dataframe
dengue_final_df

,index,year,week,number
0,0.0,2014,1,436.0
1,2.0,2014,2,479.0
2,4.0,2014,3,401.0
3,6.0,2014,4,336.0
4,8.0,2014,5,234.0
...,...,...,...,...
47,NaN,2019,48,295.0
48,NaN,2019,49,280.0
49,NaN,2019,50,257.0
50,NaN,2019,51,226.0


In [114]:
# Read from CSV
raw_population_df = pd.read_csv(
    "Datasets/Population Data/singapore-residents-by-age-group-ethnic-group-and-sex-end-june-annual.csv")
# Only get data from 2014 to 2018
population_df = raw_population_df[raw_population_df['year'].map(
    lambda x: 2014 <= int(x) <= 2018)]
# Only get total residents for each year
population_df = population_df[population_df['level_1'].map(
    lambda x: str(x) == 'Total Residents')].drop('level_1', 1).drop('level_2', 1)
population_df = population_df.astype({'year' : 'int32', 'value': 'int32'})
population_df = population_df.groupby('year').sum()
population_df = population_df.rename(columns={"value": "population"}, errors="raise")

# Display dataframe
population_df.head()

,population
year,
2014,4823625
2015,4907802
2016,4994232
2017,5089716
2018,5184977


In [165]:
# Read from CSV
raw_sunshine_df = pd.read_csv("Datasets/Sunshine Data/sunshine-duration-monthly-mean-daily-duration.csv")
# Split year_month into year and month column
raw_sunshine_df = raw_sunshine_df.rename(columns={"month": "year_month"}, errors="raise")
year_col = pd.Series([], dtype='int32')
month_col = pd.Series([], dtype='int32')
for i in range(len(raw_sunshine_df)):
    year_col[i] = int(raw_sunshine_df['year_month'][i].split('-')[0])
    month_col[i] = int(raw_sunshine_df['year_month'][i].split('-')[1])
raw_sunshine_df.insert(1, 'year', year_col)
raw_sunshine_df.insert(2, 'month', month_col)
sunshine_df = raw_sunshine_df.drop('year_month', 1)
# Display
sunshine_df.head()

,year,month,mean_sunshine_hrs
0,1982,1,5.6
1,1982,2,7.6
2,1982,3,5.1
3,1982,4,4.7
4,1982,5,5.8


In [166]:
# Read from CSV
raw_temperature_df = pd.read_csv(
    "Datasets/Surface Air Temperature Data/surface-air-temperature-monthly-mean-daily-maximum.csv")
# Split year_month into year and month column
raw_temperature_df = raw_temperature_df.rename(columns={"month": "year_month"}, errors="raise")
year_col = pd.Series([], dtype='int32')
month_col = pd.Series([], dtype='int32')
for i in range(len(raw_temperature_df)):
    year_col[i] = int(raw_temperature_df['year_month'][i].split('-')[0])
    month_col[i] = int(raw_temperature_df['year_month'][i].split('-')[1])
raw_temperature_df.insert(1, 'year', year_col)
raw_temperature_df.insert(2, 'month', month_col)
surface_air_temperature_df = raw_temperature_df.drop('year_month', 1)
# Display
surface_air_temperature_df.head()

,year,month,temp_mean_daily_max
0,1982,1,29.8
1,1982,2,32.3
2,1982,3,31.7
3,1982,4,31.4
4,1982,5,31.7


In [170]:
# Read from CSV
raw_humidity_df = pd.read_csv(
    "Datasets/Humidity Data/relative-humidity-monthly-mean.csv")
# Split year_month into year and month column
raw_humidity_df = raw_humidity_df.rename(columns={"month": "year_month"}, errors="raise")
raw_humidity_df = raw_humidity_df.rename(columns={"mean_rh": "mean_relative_humidity"}, errors="raise")
year_col = pd.Series([], dtype='int32')
month_col = pd.Series([], dtype='int32')
for i in range(len(raw_humidity_df)):
    year_col[i] = int(raw_humidity_df['year_month'][i].split('-')[0])
    month_col[i] = int(raw_humidity_df['year_month'][i].split('-')[1])
raw_humidity_df.insert(1, 'year', year_col)
raw_humidity_df.insert(2, 'month', month_col)
humidity_df = raw_humidity_df.drop('year_month', 1)
# Display
humidity_df.head()

,year,month,mean_relative_humidity
0,1982,1,81.2
1,1982,2,79.5
2,1982,3,82.3
3,1982,4,85.9
4,1982,5,83.2


Next, we shall merge the dataframes into one general dataframe **df** based on *year*, *month* and *week*.

In [171]:
# Merge dengue_df and population_df by year
population_df = pd.read_csv("Datasets/Population Data/population.csv")
df = dengue_final_df.merge(population_df, left_on='year', right_on='year').drop('index', 1)

# Create new column for cases_per_capita = number / population
# Create new column for month = (datetime(year, 1, 1) + timedelta(days=week*7)).momth
cases_per_capita = pd.Series([], dtype='float64')
month = pd.Series([], dtype='int32')
for i in range(len(df)):
    if isnan(df['number'][i]):
        cases_per_capita[i] = 0
    else:
        cases_per_capita[i] = int(df['number'][i]) / int(df['population'][i])
    year = int(df['year'][i])
    week = int(df['week'][i])
    if week > 52:
        month[i] = 12
    else:
        month[i] = (datetime(year, 1, 1) + timedelta(days=7*week)).month
df.insert(4, 'cases_per_capita', cases_per_capita)
df.insert(1, 'month', month)

# Merge sunshine and surface air temperature dataframe and humidity
df = pd.merge(df, sunshine_df, on=['year', 'month'])
df = pd.merge(df, surface_air_temperature_df, on=['year', 'month'])
df = pd.merge(df, humidity_df, on=['year', 'month'])

df.to_csv("Datasets/Singapore Population and Cases.csv", index=False)
# Display dataframe
df.head()

,year,month,week,number,population,cases_per_capita,mean_sunshine_hrs,temp_mean_daily_max,mean_relative_humidity
0,2014,1,1,436.0,5496512,0.000079,5.3,29.6,78.5
1,2014,1,2,479.0,5496512,0.000087,5.3,29.6,78.5
2,2014,1,3,401.0,5496512,0.000073,5.3,29.6,78.5
3,2014,1,4,336.0,5496512,0.000061,5.3,29.6,78.5
4,2014,2,5,234.0,5496512,0.000043,8.8,31.9,74.5


Now, we scrape the weather data from 2014 to 2019 for 5 weather stations:

In [ ]:
#weather station
# Scrape from websites
base_url = "http://www.weather.gov.sg/files/dailydata/DAILYDATA_" #S24_201911.csv

# (East) Changi = 24, (West) Tuas South = 115, (North) Khatib = 122, (South) Marina Barrage = 108
# (Central) Ang Mo Kio = 109
station_dict = {24: 'Changi', 115: 'Tuas South', 122: 'Khatib', 108: 'Marina Barrage', 109: 'Ang Mo Kio'}
stations_list = [24, 115, 122, 108, 109]

# DataFrame template
df_template = pd.DataFrame(columns = ['Station', 'Year', 'Month', 'Day', 'Daily Rainfall Total (mm)',
       'Highest 30 Min Rainfall (mm)', 'Highest 60 Min Rainfall (mm)',
       'Highest 120 Min Rainfall (mm)', 'Mean Temperature (°C)',
       'Maximum Temperature (°C)', 'Minimum Temperature (°C)',
       'Mean Wind Speed (km/h)', 'Max Wind Speed (km/h)'])
weather_df = df_template.copy()

# Scrape stations and combine into a single csv file
for station in stations_list:
    station_string = "S"+str(station) if (station>9) else "S0"+str(station)
    station_df = df_template.copy()
    for year in range(2014,2020):
        for month in range(1,13):
            month_string = str(month) if (month>9) else "0"+str(month)
            url = base_url+station_string+"_"+str(year)+month_string+".csv"
            # Get csv files to temp files, and store into DataFrame
            try:
                r = requests.get(url, allow_redirects=True)
                open('temp.csv', 'wb').write(r.content)
                station_df = station_df.append(pd.read_csv("temp.csv", encoding = "ISO-8859-1"))
                weather_df = weather_df.append(pd.read_csv("temp.csv", encoding = "ISO-8859-1"))
            except:
                print(url)
                continue
    # Store into weather data folder
    csv_filename = 'Datasets/Weather Data/' + station_dict[station] + '.csv'
    station_df.to_csv(csv_filename, index=False)


We then add the week column to the weather data

In [ ]:
#Weather station 
station_dict = {24: 'Changi', 115: 'Tuas South', 122: 'Khatib', 108: 'Marina Barrage', 109: 'Ang Mo Kio'}
stations_list = [24, 115, 122, 108, 109]

#add week column to each weather stations csv
for station in stations_list:
    week_col = pd.Series([], dtype='int32')
    df = pd.read_csv("Datasets/Weather Data/" + station_dict[station] + ".csv")
    for i in range(len(df)):
        week_col[i] = datetime(int(df['Year'][i]), int(df['Month'][i]), int(df['Day'][i])).isocalendar()[1]
    df.insert(3, 'Week', week_col)
    csv_filename = 'Datasets/Weather Data/' + station_dict[station] + '.csv'
    df.to_csv(csv_filename, index=False)

Convert PSI data to contain year, month, week and day columns:

In [141]:
#PSI
raw_psi_df = pd.read_csv("Datasets/PSI Data/psi_df_2016_2019.csv")

#split to year month and week
raw_psi_df = raw_psi_df.rename(columns={"timestamp": "year_month_week_day"}, errors="raise")
year_col = pd.Series([], dtype='int32')
month_col = pd.Series([], dtype='int32')
day_col = pd.Series([], dtype='int32')
week_col = pd.Series([], dtype='int32')
for i in range(len(raw_psi_df)):
    year_col[i] = int(raw_psi_df['year_month_week_day'][i].split('-')[0])
    month_col[i] = int(raw_psi_df['year_month_week_day'][i].split('-')[1])
    day_col[i] = int(raw_psi_df['year_month_week_day'][i].split('-')[2][0:2])
    week_col[i] = datetime(int(year_col[i]), int(month_col[i]), int(day_col[i])).isocalendar()[1]
raw_psi_df.insert(0, 'year', year_col)
raw_psi_df.insert(1, 'month', month_col)
raw_psi_df.insert(2, 'week', week_col)
raw_psi_df.insert(3, 'day', day_col)
psi_df = raw_psi_df.drop(['year_month_week_day'], 1)

#display
psi_df.head()

#find max, min avg for unique weeks in each year

columns = ['Year', 'Week', 'Max National', 'Avg National', 'Max South', 'Avg South', 'Max North', 'Avg North', 'Max East'
        , 'Avg East', 'Max Central', 'Avg Central', 'Max West', 'Avg West']
psi_max_avg_df = pd.DataFrame(columns=columns) 

index = 0

for i in range (2016, 2020):
    psi_year_df = psi_df[psi_df['year'] == i]
    unique_weeks = psi_year_df['week'].unique()
    for j in unique_weeks:
        psi_week_df = psi_year_df[psi_year_df['week'] == j]
        n = len(psi_week_df['national'].dropna())
        national_max = psi_week_df['national'].max()
        national_avg = psi_week_df['national'].sum() / n
        n = len(psi_week_df['south'].dropna())
        south_max = psi_week_df['south'].max()
        south_avg = psi_week_df['south'].sum() / n
        n = len(psi_week_df['north'].dropna())
        north_max = psi_week_df['north'].max()
        north_avg = psi_week_df['north'].sum() / n
        n = len(psi_week_df['east'].dropna())
        east_max = psi_week_df['east'].max()
        east_avg = psi_week_df['east'].sum() / n
        n = len(psi_week_df['central'].dropna())
        central_max = psi_week_df['central'].max()
        central_avg = psi_week_df['central'].sum() / n
        n = len(psi_week_df['west'].dropna())
        west_max = psi_week_df['west'].max()
        west_avg = psi_week_df['west'].sum() / n
        psi_max_avg_df.loc[index] = [i, j, national_max, national_avg, south_max, south_avg, north_max, north_avg
                                          , east_max, east_avg, central_max, central_avg, west_max, west_avg]
        index += 1
        

        
csv_filename = "Datasets/PSI Data/psi_max_avg.csv"
psi_max_avg_df.to_csv(csv_filename, index=False)       
psi_max_avg_df
        


,Year,Week,Max National,Avg National,Max South,Avg South,Max North,Avg North,Max East,Avg East,Max Central,Avg Central,Max West,Avg West
0,2016.0,5.0,47.0,47.000000,44.0,44.000000,37.0,37.000000,47.0,47.000000,47.0,47.000000,34.0,34.000000
1,2016.0,6.0,64.0,48.918519,60.0,44.970370,59.0,42.629630,64.0,47.496296,59.0,45.118519,60.0,42.525926
2,2016.0,7.0,58.0,51.679487,58.0,49.570513,56.0,47.608974,58.0,50.583333,58.0,48.608974,56.0,43.198718
3,2016.0,8.0,57.0,53.716535,54.0,49.299213,53.0,45.330709,57.0,53.708661,53.0,48.448819,52.0,47.929134
4,2016.0,9.0,55.0,52.153846,55.0,51.125000,52.0,46.086538,55.0,51.067308,54.0,48.884615,53.0,47.701923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,2019.0,41.0,67.0,58.836478,69.0,60.245283,64.0,54.716981,69.0,57.987421,68.0,55.597484,64.0,57.635220
193,2019.0,42.0,65.0,57.180124,67.0,59.180124,67.0,55.658385,66.0,58.012422,65.0,55.093168,64.0,55.739130
194,2019.0,43.0,69.0,50.276730,69.0,58.150943,68.0,55.509434,65.0,54.993711,67.0,54.974843,65.0,56.125786
195,2019.0,44.0,73.0,56.683230,75.0,64.111801,73.0,62.434783,73.0,59.975155,75.0,62.422360,74.0,59.664596


Merge 2014-2019 location dengue cluster into one csv
Add year, month, week and day columns
Split locality with '/' and '('
save as new csv

In [55]:
#merge the location data into one csv
raw_location_dengue_2014_df = pd.read_csv("Datasets/Dengue Location Cluster Data/2014-Dengue-Cluster.csv")
raw_location_dengue_2015_df = pd.read_csv("Datasets/Dengue Location Cluster Data/2015-Dengue-Cluster.csv")
raw_location_dengue_2016_df = pd.read_csv("Datasets/Dengue Location Cluster Data/2016-Dengue-Cluster.csv")
raw_location_dengue_2017_df = pd.read_csv("Datasets/Dengue Location Cluster Data/2017-Dengue-Cluster.csv", encoding='unicode_escape')
raw_location_dengue_2018_df = pd.read_csv("Datasets/Dengue Location Cluster Data/2018-Dengue-Cluster.csv")
raw_location_dengue_2019_df = pd.read_csv("Datasets/Dengue Location Cluster Data/2019-Dengue-Cluster.csv")

raw_location_dengue_df = raw_location_dengue_2014_df.append([raw_location_dengue_2015_df, raw_location_dengue_2016_df, raw_location_dengue_2017_df,
         raw_location_dengue_2018_df, raw_location_dengue_2019_df])

#save the raw csv
csv_filename = 'Datasets/Dengue Location Cluster Data/' + 'raw_location_dengue' + '.csv'
raw_location_dengue_df.to_csv(csv_filename, index=False)

raw_location_dengue_df = pd.read_csv("Datasets/Dengue Location Cluster Data/raw_location_dengue.csv")

#split year_month_day to year month and week columns
raw_location_dengue_df = raw_location_dengue_df.rename(columns={"date": "year_month_week_day"}, errors="raise")
year_col = pd.Series([], dtype='int32')
month_col = pd.Series([], dtype='int32')
day_col = pd.Series([], dtype='int32')
week_col = pd.Series([], dtype='int32')
locality_col = pd.Series([], dtype='str')
# shortcut_list = ['rd', 'jln', 'lor', 'dr', 'ave', 'st', 'pk', 'gdns', 'nth', "s'goon", 'ter']
# shortcut_dict = {'rd': 'Road', 'jln': 'Jalan', 'lor': 'Lorong', 'dr': 'Drive', 'ave': 'Avenue',
#                 'st': 'Street', 'pk' : 'Park', 'gdns' : 'Gardens', 'nth' : 'North', "s'goon" : 'Serangoon',
#                 'ter' : 'Terminal'}

for i in range(len(raw_location_dengue_df)):
#     address = []
    year_col[i] = 2000 + int(str(raw_location_dengue_df['year_month_week_day'][i])[0:2])
    month_col[i] = int(str(raw_location_dengue_df['year_month_week_day'][i])[2:4])
    day_col[i] = int(str(raw_location_dengue_df['year_month_week_day'][i])[4:])
    week_col[i] = datetime(int(year_col[i]), int(month_col[i]), int(day_col[i])).isocalendar()[1]
    #split location with '/'
    locality_col[i] = raw_location_dengue_df['locality'][i].split('/')[0]
    #split location with '('
    locality_col[i] = locality_col[i].split('(')[0]
    locality_col[i] = "".join([x if ord(x) < 128 else ' ' for x in locality_col[i]])
#     address = locality_col[i].split(' ')
#     for j in range(len(address)):
#         for shortcut in shortcut_list:
#             if(str.lower(address[j]) == shortcut):
#                 address[j] = shortcut_dict[shortcut]
#                 break
#     print(address)
#     locality_col[i] = address[0]
#     print(locality_col[i])
#     for j in range(len(address)):
#         if j != 0 and address[j] != '':
#             locality_col[i] += " " + address[j]
    
        
raw_location_dengue_df.insert(1, 'year', year_col)
raw_location_dengue_df.insert(2, 'month', month_col)
raw_location_dengue_df.insert(3, 'week', week_col)
raw_location_dengue_df.insert(4, 'day', day_col)
raw_location_dengue_df.insert(5, 'address', locality_col)
location_dengue_df = raw_location_dengue_df.drop(['year_month_week_day', 'locality'], 1)

#save
csv_filename = 'Datasets/Dengue Location Cluster Data/' + 'location_dengue' + '.csv'
location_dengue_df.to_csv(csv_filename, index=False)



Convert the buildings.json to a csv file (buildings.json contain all addresses in Singapore)

In [8]:
#path to json file
# df = pd.read_json("Datasets/buildings.json", orient="records")

# df.head()

fileInput = 'Datasets/buildings.json'
fileOutput = 'Datasets/buildings.csv'

inputFile = open(fileInput)
outputFile = open(fileOutput, 'w')
data = json.load(inputFile)
inputFile.close()
print(data)

output = csv.writer(outputFile)
output.writerow(data[0].keys()) 

for row in data:
    output.writerow(row.values())

 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Find unique locations in the location_dengue cluster csv

In [65]:
#csv of unique locations in dengue location cluster (2014 to 2019)
location_dengue_df = pd.read_csv("Datasets/Dengue Location Cluster Data/location_dengue_without_week_errors.csv")
location_set = set()
for i in range(len(location_dengue_df)):
    location_set.add(location_dengue_df['address'][i])

fileOutput = 'Datasets/unique_locations.csv'
outputFile = open(fileOutput, 'w')
output = csv.writer(outputFile)
output.writerow(['unique location'])
for location in location_set:
    output.writerow([location])
    

In [ ]:
#[IGNORE]
#add x,y coordinates to the unique locations
unique_location_df = pd.read_csv("Datasets/unique_locations.csv")
open_buildings_df = pd.read_csv("Datasets/buildings.csv")
buildings_df = open_buildings_df.drop(['POSTAL'], 1)
for i in range(len(unique_location_df)):
    for j in range(len(buildings_df)):
        if str.upper(str(unique_location_df['unique location'][i])) in str(buildings_df['ADDRESS'][j]):
            unique_location_df['X'][i] = buildings_df['X'][j]
            unique_location_df['Y'][i] = buildings_df['Y'][j]
            break

csv_filename = "Datasets/unique_locations_in_string_function_xy_new.csv"            
unique_location_df.to_csv(csv_filename, index=False)

In [ ]:
#[IGNORE]
#add x,y coordinates to the unique locations
unique_location_df = pd.read_csv("Datasets/unique_locations.csv")
open_buildings_df = pd.read_csv("Datasets/buildings.csv")
buildings_df = open_buildings_df.drop(['POSTAL'], 1)
count = 0
locality_col = pd.Series([], dtype='str')

for i in range(len(unique_location_df)):
    for j in range(len(buildings_df)):
        if fuzz.token_set_ratio(str(unique_location_df['unique location'][i]), str(buildings_df['ADDRESS'][j])) >= 60:
            count += 1
            unique_location_df['X'][i] = buildings_df['X'][j]
            unique_location_df['Y'][i] = buildings_df['Y'][j]
            locality_col[i] = buildings_df['ADDRESS'][j]
            break

print(count)
csv_filename = "Datasets/unique_locations_fuzzywuzzy_60_xy.csv"
unique_location_df.insert(3, 'address from buildings', locality_col)
unique_location_df.to_csv(csv_filename, index=False)

Put in lat and long data to the unique locations

In [82]:
fileInput = 'Datasets/unique_location_lat_long.json'
unique_location_df = pd.read_csv("Datasets/unique_locations_without_na.csv")

inputFile = open(fileInput)
data = json.load(inputFile)
inputFile.close()
lat_col = pd.Series([], dtype='float64')
long_col = pd.Series([], dtype='float64')

for i in range(len(unique_location_df)):
    if unique_location_df['unique location'][i] != 'nan':
        #print(unique_location_df['unique location'][i].strip())
        lat_col[i] = data[str(unique_location_df['unique location'][i]).strip()]['lat']
        long_col[i] = data[str(unique_location_df['unique location'][i]).strip()]['lng']

unique_location_df.insert(1, 'lat', lat_col)
unique_location_df.insert(2, 'long', long_col)

csv_filename = "Datasets/unique_locations_lat_long.csv"            
unique_location_df.to_csv(csv_filename, index=False)
    
#output.writerow(['unique location', 'lat', 'long'])
#print(data['Surin Ave\n']['lat'])

# for row in data:
#     output.writerow()
#     output.writerow(row.values())

Find nearest weather station to each unique location

In [83]:
def nearest(row, geom_union, df1, df2, geom1_col='geometry', geom2_col='geometry', src_column=None):
    """Find the nearest point and return the corresponding value from specified column."""
    # Find the geometry that is closest
    #nearest = df2[geom2_col] == nearest_points(row[geom1_col], geom_union)[1]
    # Get the corresponding value from df2 (matching is based on the geometry)
    #value = df2[nearest][src_column]
    
    nearest_geom = nearest_points(row[geom1_col], geom_union)
    nearest_point = nearest_geom[1]
    #print(nearest_point)
    value = ''
    #print(geom2_col)
    
    for i in range(len(df2)):
        if df2[geom2_col][i] == nearest_point:
            #print('hi')
            value = df2[src_column][i]
    
    return value


unique_location_lat_long_df = pd.read_csv("Datasets/unique_locations_lat_long.csv")
weatherstation_lat_long_df = pd.read_csv("Datasets/weatherstation_lat_long.csv")

unique_location_geometry = [Point(xy) for xy in zip(unique_location_lat_long_df['long'], unique_location_lat_long_df['lat'])]
weatherstation_location_geometry = [Point(xy) for xy in zip(weatherstation_lat_long_df['Long'], weatherstation_lat_long_df['Lat'])]

#weatherstations = MultiPoint(weatherstation_location_geometry)

#print(destinations)

crs = {'init': 'epsg:4326'}
unique_location_lat_long_gdf = gpd.GeoDataFrame(unique_location_lat_long_df, crs=crs, geometry=unique_location_geometry)
weatherstation_lat_long_gdf = gpd.GeoDataFrame(weatherstation_lat_long_df, crs=crs, geometry=weatherstation_location_geometry)


unary_union = weatherstation_lat_long_gdf.unary_union
#print(unary_union)
#test = unique_location_lat_long_gdf.apply(nearest, 1, geom_union=unary_union, df1=unique_location_lat_long_gdf , df2=weatherstation_lat_long_gdf, src_column='Weather station')
#print(test)

unique_location_lat_long_gdf['nearest_weather_station'] = unique_location_lat_long_gdf.apply(nearest, geom_union=unary_union, df1=unique_location_lat_long_gdf , df2=weatherstation_lat_long_gdf, src_column='Weather station', axis=1)


csv_filename = "Datasets/unique_locations_nearest_weather_station.csv"            
unique_location_lat_long_gdf.to_csv(csv_filename, index=False)

unique_location_lat_long_gdf

    

    

/usr/local/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,unique location,lat,long,geometry,nearest_weather_station
0,Chai Chee Rd,1.325311,103.923215,POINT (103.92322 1.32531),Changi
1,Jln Peradun,1.384218,103.869210,POINT (103.86921 1.38422),Ang Mo Kio
2,Pending Rd,1.376238,103.771096,POINT (103.77110 1.37624),Khatib
3,"Lor 8, 12 Geylang",1.311753,103.876993,POINT (103.87699 1.31175),Marina Barrage
4,"Tai Hwan Gr, Ter",1.356558,103.859590,POINT (103.85959 1.35656),Ang Mo Kio
...,...,...,...,...,...
2429,"Eunos Ave 1, 4, 5, 5A, 6, 7, 8A",1.322253,103.896942,POINT (103.89694 1.32225),Marina Barrage
2430,Queens Rd,1.320669,103.809345,POINT (103.80935 1.32067),Ang Mo Kio
2431,Chuan Cl,1.349226,103.861159,POINT (103.86116 1.34923),Ang Mo Kio
2432,Marsiling Dr,1.442313,103.775090,POINT (103.77509 1.44231),Khatib


Now insert the nearest_weather_station to each location in the location_dengue_without_week_errors.csv

In [84]:
location_dengue_df = pd.read_csv("Datasets/Dengue Location Cluster Data/location_dengue_without_week_errors.csv")
unique_location_with_nearest_station_df = pd.read_csv("Datasets/unique_locations_nearest_weather_station.csv")

weather_station_col = pd.Series([], dtype='str')

for i in range(len(location_dengue_df)):
    for j in range(len(unique_location_with_nearest_station_df)):
        if unique_location_with_nearest_station_df['unique location'][j].strip() == str(location_dengue_df['address'][i]).strip():
            weather_station_col[i] = unique_location_with_nearest_station_df['nearest_weather_station'][j]
            break

location_dengue_df.insert(6, 'weather station', weather_station_col)
    
csv_filename = "Datasets/Dengue Location Cluster Data/location_dengue_with_nearest_station.csv"            
location_dengue_df.to_csv(csv_filename, index=False)

location_dengue_df


,year,month,week,day,address,cases with onset in last 2 weeks,weather station
0,2014,1,1,3,Ava Rd,31,Ang Mo Kio
1,2014,1,1,3,Geylang Rd,7,Marina Barrage
2,2014,1,1,3,Bartley Rd,11,Ang Mo Kio
3,2014,1,1,3,Canberra Dr,5,Khatib
4,2014,1,1,3,Corporation Rd,6,Tuas South
...,...,...,...,...,...,...,...
20007,2019,12,52,27,Punggol Walk,2,Khatib
20008,2019,12,52,27,Rivervale Dr,2,Ang Mo Kio
20009,2019,12,52,27,Shunfu Rd,2,Ang Mo Kio
20010,2019,12,52,27,Yishun Ave 11,2,Khatib


add lat long to location_dengue_with_nearest_station

In [4]:
location_dengue_df = pd.read_csv("Datasets/Dengue Location Cluster Data/location_dengue_with_nearest_station.csv")
unique_location_with_nearest_station_df = pd.read_csv("Datasets/unique_locations_nearest_weather_station.csv")

lat_col = pd.Series([], dtype='float64')
long_col = pd.Series([], dtype='float64')
point_col = pd.Series([])

for i in range(len(location_dengue_df)):
    for j in range(len(unique_location_with_nearest_station_df)):
        if unique_location_with_nearest_station_df['unique location'][j].strip() == str(location_dengue_df['address'][i]).strip():
            lat_col[i] = unique_location_with_nearest_station_df['lat'][j]
            long_col[i] = unique_location_with_nearest_station_df['long'][j]
            point_col[i] = unique_location_with_nearest_station_df['geometry'][j]
            break

location_dengue_df.insert(7, 'lat', lat_col)
location_dengue_df.insert(8, 'long', long_col)
location_dengue_df.insert(9, 'geometry', point_col)
    
csv_filename = "Datasets/Dengue Location Cluster Data/location_dengue_with_lat_long.csv"            
location_dengue_df.to_csv(csv_filename, index=False)

location_dengue_df

                                 


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


,year,month,week,day,address,cases with onset in last 2 weeks,weather station,lat,long,geometry
0,2014,1,1,3,Ava Rd,31.0,Ang Mo Kio,1.322310,103.852480,POINT (103.8524799 1.3223104)
1,2014,1,1,3,Geylang Rd,7.0,Marina Barrage,1.313426,103.883763,POINT (103.8837629 1.3134257)
2,2014,1,1,3,Bartley Rd,11.0,Ang Mo Kio,1.344182,103.876223,POINT (103.8762229 1.3441821)
3,2014,1,1,3,Canberra Dr,5.0,Khatib,1.441736,103.828134,POINT (103.8281341 1.4417357)
4,2014,1,1,3,Corporation Rd,6.0,Tuas South,1.338075,103.717018,POINT (103.717018 1.3380754)
...,...,...,...,...,...,...,...,...,...,...
20007,2019,12,52,27,Punggol Walk,2.0,Khatib,1.405294,103.900503,POINT (103.9005026 1.4052938)
20008,2019,12,52,27,Rivervale Dr,2.0,Ang Mo Kio,1.387299,103.904567,POINT (103.9045675 1.3872989)
20009,2019,12,52,27,Shunfu Rd,2.0,Ang Mo Kio,1.351976,103.837449,POINT (103.8374493 1.3519757)
20010,2019,12,52,27,Yishun Ave 11,2.0,Khatib,1.424667,103.844617,POINT (103.8446169 1.4246671)


find avg for data with same year, same week and same address (but different dates) and put it as one row of data

In [19]:
columns = ['Year', 'Week', 'address', 'weather station', 'Max cases with onset in last 2 weeks', 'Average cases with onset in last 2 weeks', 'Min cases with onset in last 2 weeks', 'lat', 'long', 'geometry']
location_cases_max_avg_address_df = pd.DataFrame(columns=columns)

location_dengue_lat_long_df = pd.read_csv("Datasets/Dengue Location Cluster Data/location_dengue_with_lat_long.csv")

weather_stations = ['Changi', 'Ang Mo Kio', 'Khatib', 'Tuas South', 'Marina Barrage']

temp_2014_df = location_dengue_lat_long_df[location_dengue_lat_long_df['year'] == 2014]
temp_2015_df = location_dengue_lat_long_df[location_dengue_lat_long_df['year'] == 2015]
temp_2016_df = location_dengue_lat_long_df[location_dengue_lat_long_df['year'] == 2016]
temp_2017_df = location_dengue_lat_long_df[location_dengue_lat_long_df['year'] == 2017]
temp_2018_df = location_dengue_lat_long_df[location_dengue_lat_long_df['year'] == 2018]
temp_2019_df = location_dengue_lat_long_df[location_dengue_lat_long_df['year'] == 2019]

arr_2014 = temp_2014_df['week'].unique()
arr_2015 = temp_2015_df['week'].unique()
arr_2016 = temp_2016_df['week'].unique()
arr_2017 = temp_2017_df['week'].unique()
arr_2018 = temp_2018_df['week'].unique()
arr_2019 = temp_2019_df['week'].unique()

index = 0

for i in arr_2014:
    temp_df = temp_2014_df[temp_2014_df['week'] == i]
    unique_address = temp_df['address'].dropna().unique()
    for j in unique_address:
        #print(j)
        temp_address_df = temp_df[temp_df['address'] == j]
        temp_address_sr = temp_address_df['cases with onset in last 2 weeks']
        n = len(temp_address_sr.dropna())
        cases_max = temp_address_sr.max()
        cases_avg = temp_address_sr.sum() / n
        cases_min = temp_address_sr.min()
        #print(temp_address_df.values.tolist())
        values = temp_address_df.values.tolist()[0]
        weather_station = values[6]
        lat = values[7]
        long = values[8]
        point = values[9]
        location_cases_max_avg_address_df.loc[index] = [2014, i, j, weather_station, cases_max, cases_avg, cases_min, lat, long, point]
        index += 1
        
for i in arr_2015:
    temp_df = temp_2015_df[temp_2015_df['week'] == i]
    unique_address = temp_df['address'].dropna().unique()
    for j in unique_address:
        #print(j)
        temp_address_df = temp_df[temp_df['address'] == j]
        temp_address_sr = temp_address_df['cases with onset in last 2 weeks']
        n = len(temp_address_sr.dropna())
        cases_max = temp_address_sr.max()
        cases_avg = temp_address_sr.sum() / n
        cases_min = temp_address_sr.min()
        #print(temp_address_df.values.tolist())
        values = temp_address_df.values.tolist()[0]
        weather_station = values[6]
        lat = values[7]
        long = values[8]
        point = values[9]
        location_cases_max_avg_address_df.loc[index] = [2015, i, j, weather_station, cases_max, cases_avg, cases_min, lat, long, point]
        index += 1
        
for i in arr_2016:
    temp_df = temp_2016_df[temp_2016_df['week'] == i]
    unique_address = temp_df['address'].dropna().unique()
    for j in unique_address:
        #print(j)
        temp_address_df = temp_df[temp_df['address'] == j]
        temp_address_sr = temp_address_df['cases with onset in last 2 weeks']
        n = len(temp_address_sr.dropna())
        cases_max = temp_address_sr.max()
        cases_avg = temp_address_sr.sum() / n
        cases_min = temp_address_sr.min()
        #print(temp_address_df.values.tolist())
        values = temp_address_df.values.tolist()[0]
        weather_station = values[6]
        lat = values[7]
        long = values[8]
        point = values[9]
        location_cases_max_avg_address_df.loc[index] = [2016, i, j, weather_station, cases_max, cases_avg, cases_min, lat, long, point]
        index += 1
        
for i in arr_2017:
    temp_df = temp_2017_df[temp_2017_df['week'] == i]
    unique_address = temp_df['address'].dropna().unique()
    for j in unique_address:
        #print(j)
        temp_address_df = temp_df[temp_df['address'] == j]
        temp_address_sr = temp_address_df['cases with onset in last 2 weeks']
        n = len(temp_address_sr.dropna())
        cases_max = temp_address_sr.max()
        cases_avg = temp_address_sr.sum() / n
        cases_min = temp_address_sr.min()
        #print(temp_address_df.values.tolist())
        values = temp_address_df.values.tolist()[0]
        weather_station = values[6]
        lat = values[7]
        long = values[8]
        point = values[9]
        location_cases_max_avg_address_df.loc[index] = [2017, i, j, weather_station, cases_max, cases_avg, cases_min, lat, long, point]
        index += 1
        
for i in arr_2018:
    temp_df = temp_2018_df[temp_2018_df['week'] == i]
    unique_address = temp_df['address'].dropna().unique()
    for j in unique_address:
        #print(j)
        temp_address_df = temp_df[temp_df['address'] == j]
        temp_address_sr = temp_address_df['cases with onset in last 2 weeks']
        n = len(temp_address_sr.dropna())
        cases_max = temp_address_sr.max()
        cases_avg = temp_address_sr.sum() / n
        cases_min = temp_address_sr.min()
        #print(temp_address_df.values.tolist())
        values = temp_address_df.values.tolist()[0]
        weather_station = values[6]
        lat = values[7]
        long = values[8]
        point = values[9]
        location_cases_max_avg_address_df.loc[index] = [2018, i, j, weather_station, cases_max, cases_avg, cases_min, lat, long, point]
        index += 1
        
for i in arr_2019:
    temp_df = temp_2019_df[temp_2019_df['week'] == i]
    unique_address = temp_df['address'].dropna().unique()
    for j in unique_address:
        #print(j)
        temp_address_df = temp_df[temp_df['address'] == j]
        temp_address_sr = temp_address_df['cases with onset in last 2 weeks']
        n = len(temp_address_sr.dropna())
        cases_max = temp_address_sr.max()
        cases_avg = temp_address_sr.sum() / n
        cases_min = temp_address_sr.min()
        #print(temp_address_df.values.tolist())
        values = temp_address_df.values.tolist()[0]
        weather_station = values[6]
        lat = values[7]
        long = values[8]
        point = values[9]
        location_cases_max_avg_address_df.loc[index] = [2019, i, j, weather_station, cases_max, cases_avg, cases_min, lat, long, point]
        index += 1

        
csv_filename = "Datasets/Dengue Location Cluster Data/location_dengue_address_max_avg.csv"            
location_cases_max_avg_address_df.to_csv(csv_filename, index=False)
location_cases_max_avg_address_df

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:73: RuntimeWarning: invalid value encountered in double_scalars


,Year,Week,address,weather station,Max cases with onset in last 2 weeks,Average cases with onset in last 2 weeks,Min cases with onset in last 2 weeks,lat,long,geometry
0,2014,1,Ava Rd,Ang Mo Kio,31.0,31.0,31.0,1.322310,103.852480,POINT (103.8524799 1.3223104)
1,2014,1,Geylang Rd,Marina Barrage,7.0,7.0,7.0,1.313426,103.883763,POINT (103.8837629 1.3134257)
2,2014,1,Bartley Rd,Ang Mo Kio,11.0,11.0,11.0,1.344182,103.876223,POINT (103.8762229 1.3441821)
3,2014,1,Canberra Dr,Khatib,5.0,5.0,5.0,1.441736,103.828134,POINT (103.8281341 1.4417357)
4,2014,1,Corporation Rd,Tuas South,6.0,6.0,6.0,1.338075,103.717018,POINT (103.717018 1.3380754)
...,...,...,...,...,...,...,...,...,...,...
14820,2019,52,Marine Cres,Marina Barrage,2.0,2.0,2.0,1.305336,103.912028,POINT (103.9120281 1.3053361)
14821,2019,52,Punggol Walk,Khatib,2.0,2.0,2.0,1.405294,103.900503,POINT (103.9005026 1.4052938)
14822,2019,52,Shunfu Rd,Ang Mo Kio,2.0,2.0,2.0,1.351976,103.837449,POINT (103.8374493 1.3519757)
14823,2019,52,Yishun Ave 11,Khatib,2.0,2.0,2.0,1.424667,103.844617,POINT (103.8446169 1.4246671)


now append weather data

In [34]:
location_dengue_max_avg_df = pd.read_csv("Datasets/Dengue Location Cluster Data/location_dengue_address_max_avg.csv")

amk_weather_df = pd.read_csv("Datasets/Weather Data/Weekly Ang Mo Kio.csv")
changi_weather_df = pd.read_csv("Datasets/Weather Data/Weekly Changi.csv")
khatib_weather_df = pd.read_csv("Datasets/Weather Data/Weekly Khatib.csv")
mb_weather_df = pd.read_csv("Datasets/Weather Data/Weekly Marina Barrage.csv")
ts_weather_df = pd.read_csv("Datasets/Weather Data/Weekly Tuas South.csv")

# rows_del = location_dengue_max_avg_df[(location_dengue_max_avg_df['Year'] == 2014) & (location_dengue_max_avg_df['Week'] == 1)].index
# # Delete these row indexes from dataFrame
# location_dengue_max_avg_df.drop(rows_del , inplace=True)
# rows_del = location_dengue_max_avg_df[(location_dengue_max_avg_df['Year'] == 2014) & (location_dengue_max_avg_df['Week'] == 2)].index
# location_dengue_max_avg_df.drop(rows_del , inplace=True)
# rows_del = location_dengue_max_avg_df[(location_dengue_max_avg_df['Year'] == 2014) & (location_dengue_max_avg_df['Week'] == 3)].index
# location_dengue_max_avg_df.drop(rows_del , inplace=True)
# rows_del = location_dengue_max_avg_df[(location_dengue_max_avg_df['Year'] == 2014) & (location_dengue_max_avg_df['Week'] == 4)].index
# location_dengue_max_avg_df.drop(rows_del , inplace=True)
# rows_del = location_dengue_max_avg_df[(location_dengue_max_avg_df['Year'] == 2014) & (location_dengue_max_avg_df['Week'] == 5)].index
# location_dengue_max_avg_df.drop(rows_del , inplace=True)
# rows_del = location_dengue_max_avg_df[(location_dengue_max_avg_df['Year'] == 2014) & (location_dengue_max_avg_df['Week'] == 6)].index
# location_dengue_max_avg_df.drop(rows_del , inplace=True)
# rows_del = location_dengue_max_avg_df[(location_dengue_max_avg_df['Year'] == 2014) & (location_dengue_max_avg_df['Week'] == 7)].index
# location_dengue_max_avg_df.drop(rows_del , inplace=True)
# rows_del = location_dengue_max_avg_df[(location_dengue_max_avg_df['Year'] == 2014) & (location_dengue_max_avg_df['Week'] == 8)].index
# location_dengue_max_avg_df.drop(rows_del , inplace=True)

# rows_del = location_dengue_max_avg_df[(location_dengue_max_avg_df['Year'] == 2019) & (location_dengue_max_avg_df['Week'] >= 14) & ((location_dengue_max_avg_df['Week'] < 14))].index

# csv_filename = "Datasets/Dengue Location Cluster Data/location_dengue_testtest.csv"            
# location_dengue_max_avg_df.to_csv(csv_filename, index=False)

# location_dengue_max_avg_df = pd.read_csv("Datasets/Dengue Location Cluster Data/location_dengue_testtest.csv")

# 'Max Daily Rainfall Total (mm)', 'Min Daily Rainfall Total (mm)',
#                'Average Daily Rainfall Total (mm)', 'Highest 30 Min Rainfall (mm)', 'Highest 60 Min Rainfall (mm)',
#                   'Highest 120 Min Rainfall (mm)', 'Mean Temperature (°C)', 'Maximum Temperature (°C)',
#                       'Minimum Temperature (°C)', 'Mean Wind Speed (km/h)', 'Max Wind Speed (km/h)'
            
#             'Average Daily Rainfall Total (mm)', 
#                         'Highest 60 Min Rainfall (mm)', 
#                         'Mean Temperature (°C)', 
#                         'Range of Temperature (°C)',
#                         'Mean Wind Speed (km/h)', 
#                         'mean_sunshine_hrs', 
#                         'mean_relative_humidity',
#                         'Cases in week'
            
avg_daily_rainfall_total_col = pd.Series([], dtype='float64')
highest_60_min_rainfall_col = pd.Series([], dtype='float64')
mean_temperature_col = pd.Series([], dtype='float64')
range_temperature_col = pd.Series([], dtype='float64')
mean_wind_speed_col = pd.Series([], dtype='float64')
historical_cases_week_col = pd.Series([], dtype='int32')


for i in range(len(location_dengue_max_avg_df)):
    if location_dengue_max_avg_df['Year'][i] == 2014 and location_dengue_max_avg_df['Week'][i] <= 5:
        continue
    if location_dengue_max_avg_df['Year'][i] == 2019 and location_dengue_max_avg_df['Week'][i] < 19 and location_dengue_max_avg_df['Week'][i] >= 14:
        continue
        
    weather_station = location_dengue_max_avg_df['weather station'][i]
    year = location_dengue_max_avg_df['Year'][i] - 1
    year_df = amk_weather_df[amk_weather_df['Year'] == year]
    year_week_df = year_df['Week']
    max_week = year_week_df.max()
    
    avg_daily_rainfall_total = 0
    highest_60_min_rainfall = 0
    mean_temperature = 0
    range_temperature = 0
    mean_wind_speed = 0
    historical_cases_week = 0
    
    
    if str(weather_station).strip() == 'Ang Mo Kio':
        week = location_dengue_max_avg_df['Week'][i]
        historical_week = week - 5
        for i in range(0, 4)
            if historical_week+i <= 0:
                weather_week = max_week + (historical_week+i)
                weather_year = year
            else:
                weather_week = historical_week + i
                weather year = year + 1
            weather_df = amk_weather_df[(amk_weather_df['Year'] == weather_year) & (amk_weather_df['Week'] == weather_week)]
            location_df = location_dengue_max_avg_df[(location_dengue_max_avg_df['Year'] == weather_year) & (location_dengue_max_avg_df['Week'] == weather_week)]
            avg_daily_rainfall_total += weather_df.values.tolist()[0][4]
            highest_60_min_rainfall += weather_df.values.tolist()[0][6]
            mean_temperature += weather_df.values.tolist()[0][8]
            range_temperature += weather_df.values.tolist()[0][9] - weather_df.values.tolist()[0][10]
            mean_wind_speed += weather_df.values.tolist()[0][11]
            if i > 0:
                historical_cases_week += location_df.values.tolist()[0][]
            
            
#     if str(weather_station).strip() == 'Changi':
        
#     if str(weather_station).strip() == 'Khatib':
        
#     if str(weather_station).strip() == 'Marina Barrage':
    
#     if str(weather_station).strip() == 'Tuas South':

#df = dengue_final_df.merge(population_df, left_on='year', right_on='year')

Find min, max, avg for data with same week AND same weather station (ie ang mo kio) (but different dates) and put it as one row of data 

In [125]:
location_dengue_df = pd.read_csv("Datasets/Dengue Location Cluster Data/location_dengue_with_nearest_station.csv")
columns = ['year', 'month', 'week', 'day', 'weather station', 'cases with onset in last 2 weeks']
location_cases_df = pd.DataFrame(columns=columns) 

for i in range(len(location_dengue_df)):
    str(location_dengue_df['cases with onset in last 2 weeks'][i]).encode("ascii", "ignore")

temp_2014_df = location_dengue_df[location_dengue_df['year'] == 2014]
temp_2015_df = location_dengue_df[location_dengue_df['year'] == 2015]
temp_2016_df = location_dengue_df[location_dengue_df['year'] == 2016]
temp_2017_df = location_dengue_df[location_dengue_df['year'] == 2017]
temp_2018_df = location_dengue_df[location_dengue_df['year'] == 2018]
temp_2019_df = location_dengue_df[location_dengue_df['year'] == 2019]

weather_stations = ['Changi', 'Ang Mo Kio', 'Khatib', 'Tuas South', 'Marina Barrage']

#arr14 = temp_2014_df['day'].unique()

index = 0

#for 2014, sum those with same month && same week && same day && same weather station
for i in range(1, 13):
    temp_month_df = temp_2014_df[temp_2014_df['month'] == i]
    arr14 = temp_month_df['day'].unique()
    for j in arr14:
        temp_df = temp_month_df[temp_month_df['day'] == j]
        for k in weather_stations:
            temp_station_df = temp_df[temp_df['weather station'] == k]
            num = temp_station_df['cases with onset in last 2 weeks'].dropna().astype('int32').sum()
            week = datetime(2014, i, j).isocalendar()[1]
            location_cases_df.loc[index] = [2014, i, week, j, k, num]
            index += 1
            
#for 2015, sum those with same month && same week && same day && same weather station
for i in range(1, 13):
    temp_month_df = temp_2015_df[temp_2015_df['month'] == i]
    arr15 = temp_month_df['day'].unique()
    for j in arr15:
        temp_df = temp_month_df[temp_month_df['day'] == j]
        for k in weather_stations:
            temp_station_df = temp_df[temp_df['weather station'] == k]
            num = temp_station_df['cases with onset in last 2 weeks'].dropna().astype('int32').sum()
            week = datetime(2015, i, j).isocalendar()[1]
            location_cases_df.loc[index] = [2015, i, week, j, k, num]
            index += 1

#for 2016, sum those with same month && same week && same day && same weather station
for i in range(1, 13):
    temp_month_df = temp_2016_df[temp_2016_df['month'] == i]
    arr16 = temp_month_df['day'].unique()
    for j in arr16:
        temp_df = temp_month_df[temp_month_df['day'] == j]
        for k in weather_stations:
            temp_station_df = temp_df[temp_df['weather station'] == k]
            num = temp_station_df['cases with onset in last 2 weeks'].dropna().astype('int32').sum()
            week = datetime(2016, i, j).isocalendar()[1]
            location_cases_df.loc[index] = [2016, i, week, j, k, num]
            index += 1

#for 2017, sum those with same month && same week && same day && same weather station
for i in range(1, 13):
    temp_month_df = temp_2017_df[temp_2017_df['month'] == i]
    arr17 = temp_month_df['day'].unique()
    for j in arr17:
        temp_df = temp_month_df[temp_month_df['day'] == j]
        for k in weather_stations:
            temp_station_df = temp_df[temp_df['weather station'] == k]
            num = temp_station_df['cases with onset in last 2 weeks'].dropna().astype('int32').sum()
            week = datetime(2017, i, j).isocalendar()[1]
            location_cases_df.loc[index] = [2017, i, week, j, k, num]
            index += 1
            
#for 2018, sum those with same month && same week && same day && same weather station
for i in range(1, 13):
    temp_month_df = temp_2018_df[temp_2018_df['month'] == i]
    arr18 = temp_month_df['day'].unique()
    for j in arr18:
        temp_df = temp_month_df[temp_month_df['day'] == j]
        for k in weather_stations:
            temp_station_df = temp_df[temp_df['weather station'] == k]
            num = temp_station_df['cases with onset in last 2 weeks'].dropna().astype('int32').sum()
            week = datetime(2018, i, j).isocalendar()[1]
            location_cases_df.loc[index] = [2018, i, week, j, k, num]
            index += 1
            
#for 2019, sum those with same month && same week && same day && same weather station
for i in range(1, 13):
    temp_month_df = temp_2019_df[temp_2019_df['month'] == i]
    arr19 = temp_month_df['day'].unique()
    for j in arr19:
        temp_df = temp_month_df[temp_month_df['day'] == j]
        for k in weather_stations:
            temp_station_df = temp_df[temp_df['weather station'] == k]
            num = temp_station_df['cases with onset in last 2 weeks'].dropna().astype('int32').sum()
            week = datetime(2019, i, j).isocalendar()[1]
            location_cases_df.loc[index] = [2019, i, week, j, k, num]
            index += 1

csv_filename = "Datasets/Dengue Location Cluster Data/location_dengue_sum.csv"            
location_cases_df.to_csv(csv_filename, index=False)

location_cases_df
            


,year,month,week,day,weather station,cases with onset in last 2 weeks
0,2014,1,1,3,Changi,20
1,2014,1,1,3,Ang Mo Kio,87
2,2014,1,1,3,Khatib,20
3,2014,1,1,3,Tuas South,7
4,2014,1,1,3,Marina Barrage,21
...,...,...,...,...,...,...
2760,2019,12,52,27,Changi,17
2761,2019,12,52,27,Ang Mo Kio,100
2762,2019,12,52,27,Khatib,75
2763,2019,12,52,27,Tuas South,12


In [130]:
#max, min, avg

columns = ['Year', 'Week', 'weather station', 'Max cases with onset in last 2 weeks', 'Average cases with onset in last 2 weeks', 'Min cases with onset in last 2 weeks']
location_cases_max_avg_df = pd.DataFrame(columns=columns)

location_dengue_sum_df = pd.read_csv("Datasets/Dengue Location Cluster Data/location_dengue_sum_final.csv")

weather_stations = ['Changi', 'Ang Mo Kio', 'Khatib', 'Tuas South', 'Marina Barrage']

temp_2014_df = location_dengue_sum_df[location_dengue_sum_df['year'] == 2014]
temp_2015_df = location_dengue_sum_df[location_dengue_sum_df['year'] == 2015]
temp_2016_df = location_dengue_sum_df[location_dengue_sum_df['year'] == 2016]
temp_2017_df = location_dengue_sum_df[location_dengue_sum_df['year'] == 2017]
temp_2018_df = location_dengue_sum_df[location_dengue_sum_df['year'] == 2018]
temp_2019_df = location_dengue_sum_df[location_dengue_sum_df['year'] == 2019]

arr_2014 = temp_2014_df['week'].unique()
arr_2015 = temp_2015_df['week'].unique()
arr_2016 = temp_2016_df['week'].unique()
arr_2017 = temp_2017_df['week'].unique()
arr_2018 = temp_2018_df['week'].unique()
arr_2019 = temp_2019_df['week'].unique()

index = 0

for i in arr_2014:
    temp_df = temp_2014_df[temp_2014_df['week'] == i]
    for j in weather_stations:
        temp_station_df = temp_df[temp_df['weather station'] == j]
        temp_station_sr = temp_station_df['cases with onset in last 2 weeks']
        n = len(temp_station_sr.dropna())
        cases_max = temp_station_sr.max()
        cases_avg = temp_station_sr.sum() / n
        cases_min = temp_station_sr.min()
        location_cases_max_avg_df.loc[index] = [2014, i, j, cases_max, cases_avg, cases_min]
        index += 1

for i in arr_2015:
    temp_df = temp_2015_df[temp_2015_df['week'] == i]
    for j in weather_stations:
        temp_station_df = temp_df[temp_df['weather station'] == j]
        temp_station_sr = temp_station_df['cases with onset in last 2 weeks']
        n = len(temp_station_sr.dropna())
        cases_max = temp_station_sr.max()
        cases_avg = temp_station_sr.sum() / n
        cases_min = temp_station_sr.min()
        location_cases_max_avg_df.loc[index] = [2015, i, j, cases_max, cases_avg, cases_min]
        index += 1
        
for i in arr_2016:
    temp_df = temp_2016_df[temp_2016_df['week'] == i]
    for j in weather_stations:
        temp_station_df = temp_df[temp_df['weather station'] == j]
        temp_station_sr = temp_station_df['cases with onset in last 2 weeks']
        n = len(temp_station_sr.dropna())
        cases_max = temp_station_sr.max()
        cases_avg = temp_station_sr.sum() / n
        cases_min = temp_station_sr.min()
        location_cases_max_avg_df.loc[index] = [2016, i, j, cases_max, cases_avg, cases_min]
        index += 1
        
for i in arr_2017:
    temp_df = temp_2017_df[temp_2017_df['week'] == i]
    for j in weather_stations:
        temp_station_df = temp_df[temp_df['weather station'] == j]
        temp_station_sr = temp_station_df['cases with onset in last 2 weeks']
        n = len(temp_station_sr.dropna())
        cases_max = temp_station_sr.max()
        cases_avg = temp_station_sr.sum() / n
        cases_min = temp_station_sr.min()
        location_cases_max_avg_df.loc[index] = [2017, i, j, cases_max, cases_avg, cases_min]
        index += 1
        
for i in arr_2018:
    temp_df = temp_2018_df[temp_2018_df['week'] == i]
    for j in weather_stations:
        temp_station_df = temp_df[temp_df['weather station'] == j]
        temp_station_sr = temp_station_df['cases with onset in last 2 weeks']
        n = len(temp_station_sr.dropna())
        cases_max = temp_station_sr.max()
        cases_avg = temp_station_sr.sum() / n
        cases_min = temp_station_sr.min()
        location_cases_max_avg_df.loc[index] = [2018, i, j, cases_max, cases_avg, cases_min]
        index += 1
    
for i in arr_2019:
    temp_df = temp_2019_df[temp_2019_df['week'] == i]
    for j in weather_stations:
        temp_station_df = temp_df[temp_df['weather station'] == j]
        temp_station_sr = temp_station_df['cases with onset in last 2 weeks']
        n = len(temp_station_sr.dropna())
        cases_max = temp_station_sr.max()
        cases_avg = temp_station_sr.sum() / n
        cases_min = temp_station_sr.min()
        location_cases_max_avg_df.loc[index] = [2019, i, j, cases_max, cases_avg, cases_min]
        index += 1

        
csv_filename = "Datasets/Dengue Location Cluster Data/location_dengue_max_min_avg.csv"            
location_cases_max_avg_df.to_csv(csv_filename, index=False)

location_cases_max_avg_df

,Year,Week,weather station,Max cases with onset in last 2 weeks,Average cases with onset in last 2 weeks,Min cases with onset in last 2 weeks
0,2014,1,Changi,20,20.0,20
1,2014,1,Ang Mo Kio,87,87.0,87
2,2014,1,Khatib,20,20.0,20
3,2014,1,Tuas South,7,7.0,7
4,2014,1,Marina Barrage,21,21.0,21
...,...,...,...,...,...,...
1495,2019,52,Changi,17,17.0,17
1496,2019,52,Ang Mo Kio,100,100.0,100
1497,2019,52,Khatib,75,75.0,75
1498,2019,52,Tuas South,12,12.0,12


Now we merge the data with location-based weather and separate them into different csv for different weather station

for amk:

In [132]:
columns = ['Year', 'Week', 'weather station', 'Max cases with onset in last 2 weeks', 'Average cases with onset in last 2 weeks', 'Min cases with onset in last 2 weeks']
location_cases_amk_df = pd.DataFrame(columns=columns)


weather_amk_df = pd.read_csv("Datasets/Weather Data/Weekly Ang Mo Kio.csv")


location_dengue_max_avg_df = pd.read_csv("Datasets/Dengue Location Cluster Data/location_dengue_max_min_avg_final.csv")

#for ang mo kio
index = 0
for i in range (2014, 2020):
    location_year_df = location_dengue_max_avg_df[(location_dengue_max_avg_df['Year'] == i) & (location_dengue_max_avg_df['weather station'] == 'Ang Mo Kio')]
    weather_year_df = weather_amk_df[weather_amk_df['Year'] == i]
    for j in range(1,54):
        location_week_df = location_year_df[(location_year_df['Week'] == j)]
        weather_week_df = weather_year_df[weather_year_df['Week'] == j]
        row = []
        if len(weather_week_df) == 0:
            continue
        if len(location_week_df) == 0:
            row = [i, j, 'Ang Mo Kio', np.nan, np.nan, np.nan]
        else:
            row = location_week_df.values.tolist()[0]
        
        location_cases_amk_df.loc[index] = row
        index += 1
        
del weather_amk_df['Year']
del weather_amk_df['Week']
amk_result = pd.concat([location_cases_amk_df, weather_amk_df], axis=1).reindex(location_cases_amk_df.index)
amk_result.to_csv("Datasets/Model Training merged data/Ang Mo Kio Weather and Cases.csv", index=False)







aggregate amk:

In [64]:
# 'Max Daily Rainfall Total (mm)', 'Min Daily Rainfall Total (mm)',
#                'Average Daily Rainfall Total (mm)', 'Highest 30 Min Rainfall (mm)', 'Highest 60 Min Rainfall (mm)',
#                   'Highest 120 Min Rainfall (mm)', 'Mean Temperature (°C)', 'Maximum Temperature (°C)',
#                       'Minimum Temperature (°C)', 'Mean Wind Speed (km/h)', 'Max Wind Speed (km/h)'
            
#             'Average Daily Rainfall Total (mm)', 
#                         'Highest 60 Min Rainfall (mm)', 
#                         'Mean Temperature (°C)', 
#                         'Range of Temperature (°C)',
#                         'Mean Wind Speed (km/h)', 
#                         'mean_sunshine_hrs', 
#                         'mean_relative_humidity',
#                         'Cases in week'


amk_weather_df = pd.read_csv("Datasets/Weather Data/Weekly Ang Mo Kio.csv")


location_dengue_max_avg_df = pd.read_csv("Datasets/Dengue Location Cluster Data/location_dengue_max_min_avg_final.csv")
amk_location_dengue = location_dengue_max_avg_df[location_dengue_max_avg_df['weather station'] == 'Ang Mo Kio']
amk_location_dengue.to_csv("Datasets/Model Training merged data/Ang Mo Kio without weather.csv", index=False)
amk_location_dengue = pd.read_csv("Datasets/Model Training merged data/Ang Mo Kio without weather.csv")

avg_daily_rainfall_total_col = pd.Series([], dtype='float64')
highest_60_min_rainfall_col = pd.Series([], dtype='float64')
mean_temperature_col = pd.Series([], dtype='float64')
range_temperature_col = pd.Series([], dtype='float64')
mean_wind_speed_col = pd.Series([], dtype='float64')
historical_cases_week_col = pd.Series([], dtype='int32')

for i in range(len(amk_location_dengue)):
    if amk_location_dengue['Year'][i] == 2014 and amk_location_dengue['Week'][i] <= 5:
        avg_daily_rainfall_total_col[i] = np.nan
        highest_60_min_rainfall_col[i] = np.nan
        mean_temperature_col[i] = np.nan
        range_temperature_col[i] = np.nan
        mean_wind_speed_col[i] = np.nan
        historical_cases_week_col[i] = np.nan
        continue
    if amk_location_dengue['Year'][i] == 2019 and amk_location_dengue['Week'][i] < 19 and amk_location_dengue['Week'][i] >= 14:
        avg_daily_rainfall_total_col[i] = np.nan
        highest_60_min_rainfall_col[i] = np.nan
        mean_temperature_col[i] = np.nan
        range_temperature_col[i] = np.nan
        mean_wind_speed_col[i] = np.nan
        historical_cases_week_col[i] = np.nan
        continue
    
    #find number of weeks in previous year in case
    year = amk_location_dengue['Year'][i] - 1
    year_df = amk_weather_df[amk_weather_df['Year'] == year]
    year_week_df = year_df['Week']
    max_week = year_week_df.max()
    
    avg_daily_rainfall_total = 0
    highest_60_min_rainfall = 0
    mean_temperature = 0
    range_temperature = 0
    mean_wind_speed = 0
    historical_cases_week = 0
    

    week = amk_location_dengue['Week'][i]
    historical_week = week - 5
    for j in range(0, 4):
        if historical_week+j <= 0:
            weather_week = max_week + (historical_week+j)
            weather_year = year
        else:
            weather_week = historical_week + j
            weather_year = year + 1
        weather_df = amk_weather_df[(amk_weather_df['Year'] == weather_year) & (amk_weather_df['Week'] == weather_week)]
        location_df = amk_location_dengue[(amk_location_dengue['Year'] == weather_year) & (amk_location_dengue['Week'] == weather_week)]
        avg_daily_rainfall_total += weather_df.values.tolist()[0][4]
        highest_60_min_rainfall += weather_df.values.tolist()[0][6]
        mean_temperature += weather_df.values.tolist()[0][8]
        range_temperature += weather_df.values.tolist()[0][9] - weather_df.values.tolist()[0][10]
        mean_wind_speed += weather_df.values.tolist()[0][11]
        if j > 0:
            if len(location_df) == 0:
                historical_cases_week += 0
            else:
                historical_cases_week += location_df.values.tolist()[0][4]
                
    avg_daily_rainfall_total_col[i] = avg_daily_rainfall_total / 4
    highest_60_min_rainfall_col[i] = highest_60_min_rainfall / 4
    mean_temperature_col[i] = mean_temperature / 4
    range_temperature_col[i] = range_temperature / 4
    mean_wind_speed_col[i] = mean_wind_speed / 4
    historical_cases_week_col[i] = historical_cases_week / 3
    

    
amk_location_dengue.insert(6, 'avg_daily_rainfall_total', avg_daily_rainfall_total_col)
amk_location_dengue.insert(7, 'highest_60_min_rainfall', highest_60_min_rainfall_col)
amk_location_dengue.insert(8, 'mean_temperature', mean_temperature_col)
amk_location_dengue.insert(9, 'range_temperature', range_temperature_col)
amk_location_dengue.insert(10, 'mean_wind_speed', mean_wind_speed_col)
amk_location_dengue.insert(11, 'historical_cases_week', historical_cases_week_col)

amk_location_dengue.to_csv("Datasets/Model Training merged data/Ang Mo Kio Aggregate.csv", index=False)
amk_location_dengue





,Year,Week,weather station,Max cases with onset in last 2 weeks,Average cases with onset in last 2 weeks,Min cases with onset in last 2 weeks,avg_daily_rainfall_total,highest_60_min_rainfall,mean_temperature,range_temperature,mean_wind_speed,historical_cases_week
0,2014,1,Ang Mo Kio,87,87.000000,87,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,2,Ang Mo Kio,91,79.500000,68,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,3,Ang Mo Kio,95,91.500000,88,NaN,NaN,NaN,NaN,NaN,NaN
3,2014,4,Ang Mo Kio,69,69.000000,69,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,5,Ang Mo Kio,57,54.666667,50,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
295,2019,48,Ang Mo Kio,143,143.000000,143,7.865714,25.85,28.008571,10.575,10.707143,141.000000
296,2019,49,Ang Mo Kio,146,146.000000,146,7.028571,25.85,27.857143,10.775,11.439286,149.333333
297,2019,50,Ang Mo Kio,111,111.000000,111,7.039286,22.70,27.822619,10.650,12.303571,149.000000
298,2019,51,Ang Mo Kio,93,93.000000,93,4.451786,12.55,27.435119,9.050,13.610714,148.333333


In [151]:
amk_df = pd.read_csv("Datasets/Model Training merged data/Ang Mo Kio Weather and Cases.csv")
psi_max_avg_df = pd.read_csv("Datasets/PSI Data/psi_max_avg.csv")

#columns = ['Year', 'Week', 'Max National', 'Avg National', 'Max South', 'Avg South', 'Max North', 'Avg North', 'Max East'
        #, 'Avg East', 'Max Central', 'Avg Central', 'Max West', 'Avg West']
columns = ['Year', 'Week', 'Max PSI Central', 'Avg PSI Central']
psi_max_avg_amk_df = pd.DataFrame(columns=columns) 

index = 0

for i in range (2014, 2020):
    psi_year_df = psi_max_avg_df[(psi_max_avg_df['Year'] == i)]
    amk_year_df = amk_df[amk_df['Year'] == i]
    for j in range(1,54):
        psi_week_df = psi_year_df[(psi_year_df['Week'] == j)]
        amk_week_df = amk_year_df[amk_year_df['Week'] == j]
        row = []
        if len(amk_week_df) == 0:
            continue
        if len(psi_week_df) == 0:
            row = [i, j, np.nan, np.nan]
        else:
            values = psi_week_df.values.tolist()[0]
            row = [i, j, values[10], values[11]]
            
        psi_max_avg_amk_df.loc[index] = row
        index += 1
        
del psi_max_avg_amk_df['Year']
del psi_max_avg_amk_df['Week']
amk_result = pd.concat([amk_df, psi_max_avg_amk_df], axis=1).reindex(amk_df.index)
amk_result.to_csv("Datasets/Model Training merged data/Ang Mo Kio Weather PSI and Cases.csv", index=False)

for changi:

In [133]:
columns = ['Year', 'Week', 'weather station', 'Max cases with onset in last 2 weeks', 'Average cases with onset in last 2 weeks', 'Min cases with onset in last 2 weeks']

location_cases_changi_df = pd.DataFrame(columns=columns)

weather_changi_df = pd.read_csv("Datasets/Weather Data/Weekly Changi.csv")

location_dengue_max_avg_df = pd.read_csv("Datasets/Dengue Location Cluster Data/location_dengue_max_min_avg_final.csv")

#for changi
index = 0
for i in range (2014, 2020):
    location_year_df = location_dengue_max_avg_df[(location_dengue_max_avg_df['Year'] == i) & (location_dengue_max_avg_df['weather station'] == 'Changi')]
    weather_year_df = weather_changi_df[weather_changi_df['Year'] == i]
    for j in range(1,54):
        location_week_df = location_year_df[(location_year_df['Week'] == j)]
        weather_week_df = weather_year_df[weather_year_df['Week'] == j]
        row = []
        if len(weather_week_df) == 0:
            continue
        if len(location_week_df) == 0:
            row = [i, j, 'Changi', np.nan, np.nan, np.nan]
        else:
            row = location_week_df.values.tolist()[0]
        
        location_cases_changi_df.loc[index] = row
        index += 1
        
del weather_changi_df['Year']
del weather_changi_df['Week']
changi_result = pd.concat([location_cases_changi_df, weather_changi_df], axis=1).reindex(location_cases_changi_df.index)
changi_result.to_csv("Datasets/Model Training merged data/Changi Weather and Cases.csv", index=False)






aggregate changi

In [65]:
amk_weather_df = pd.read_csv("Datasets/Weather Data/Weekly Changi.csv")


location_dengue_max_avg_df = pd.read_csv("Datasets/Dengue Location Cluster Data/location_dengue_max_min_avg_final.csv")
amk_location_dengue = location_dengue_max_avg_df[location_dengue_max_avg_df['weather station'] == 'Changi']
amk_location_dengue.to_csv("Datasets/Model Training merged data/Changi without weather.csv", index=False)
amk_location_dengue = pd.read_csv("Datasets/Model Training merged data/Changi without weather.csv")

avg_daily_rainfall_total_col = pd.Series([], dtype='float64')
highest_60_min_rainfall_col = pd.Series([], dtype='float64')
mean_temperature_col = pd.Series([], dtype='float64')
range_temperature_col = pd.Series([], dtype='float64')
mean_wind_speed_col = pd.Series([], dtype='float64')
historical_cases_week_col = pd.Series([], dtype='int32')

for i in range(len(amk_location_dengue)):
    if amk_location_dengue['Year'][i] == 2014 and amk_location_dengue['Week'][i] <= 5:
        avg_daily_rainfall_total_col[i] = np.nan
        highest_60_min_rainfall_col[i] = np.nan
        mean_temperature_col[i] = np.nan
        range_temperature_col[i] = np.nan
        mean_wind_speed_col[i] = np.nan
        historical_cases_week_col[i] = np.nan
        continue
    if amk_location_dengue['Year'][i] == 2019 and amk_location_dengue['Week'][i] < 19 and amk_location_dengue['Week'][i] >= 14:
        avg_daily_rainfall_total_col[i] = np.nan
        highest_60_min_rainfall_col[i] = np.nan
        mean_temperature_col[i] = np.nan
        range_temperature_col[i] = np.nan
        mean_wind_speed_col[i] = np.nan
        historical_cases_week_col[i] = np.nan
        continue
    
    #find number of weeks in previous year in case
    year = amk_location_dengue['Year'][i] - 1
    year_df = amk_weather_df[amk_weather_df['Year'] == year]
    year_week_df = year_df['Week']
    max_week = year_week_df.max()
    
    avg_daily_rainfall_total = 0
    highest_60_min_rainfall = 0
    mean_temperature = 0
    range_temperature = 0
    mean_wind_speed = 0
    historical_cases_week = 0
    

    week = amk_location_dengue['Week'][i]
    historical_week = week - 5
    for j in range(0, 4):
        if historical_week+j <= 0:
            weather_week = max_week + (historical_week+j)
            weather_year = year
        else:
            weather_week = historical_week + j
            weather_year = year + 1
        weather_df = amk_weather_df[(amk_weather_df['Year'] == weather_year) & (amk_weather_df['Week'] == weather_week)]
        location_df = amk_location_dengue[(amk_location_dengue['Year'] == weather_year) & (amk_location_dengue['Week'] == weather_week)]
        avg_daily_rainfall_total += weather_df.values.tolist()[0][4]
        highest_60_min_rainfall += weather_df.values.tolist()[0][6]
        mean_temperature += weather_df.values.tolist()[0][8]
        range_temperature += weather_df.values.tolist()[0][9] - weather_df.values.tolist()[0][10]
        mean_wind_speed += weather_df.values.tolist()[0][11]
        if j > 0:
            if len(location_df) == 0:
                historical_cases_week += 0
            else:
                historical_cases_week += location_df.values.tolist()[0][4]
                
    avg_daily_rainfall_total_col[i] = avg_daily_rainfall_total / 4
    highest_60_min_rainfall_col[i] = highest_60_min_rainfall / 4
    mean_temperature_col[i] = mean_temperature / 4
    range_temperature_col[i] = range_temperature / 4
    mean_wind_speed_col[i] = mean_wind_speed / 4
    historical_cases_week_col[i] = historical_cases_week / 3
    

    
amk_location_dengue.insert(6, 'avg_daily_rainfall_total', avg_daily_rainfall_total_col)
amk_location_dengue.insert(7, 'highest_60_min_rainfall', highest_60_min_rainfall_col)
amk_location_dengue.insert(8, 'mean_temperature', mean_temperature_col)
amk_location_dengue.insert(9, 'range_temperature', range_temperature_col)
amk_location_dengue.insert(10, 'mean_wind_speed', mean_wind_speed_col)
amk_location_dengue.insert(11, 'historical_cases_week', historical_cases_week_col)

amk_location_dengue.to_csv("Datasets/Model Training merged data/Changi Aggregate.csv", index=False)
amk_location_dengue


,Year,Week,weather station,Max cases with onset in last 2 weeks,Average cases with onset in last 2 weeks,Min cases with onset in last 2 weeks,avg_daily_rainfall_total,highest_60_min_rainfall,mean_temperature,range_temperature,mean_wind_speed,historical_cases_week
0,2014,1,Changi,20,20.000000,20,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,2,Changi,21,17.500000,14,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,3,Changi,11,10.000000,9,NaN,NaN,NaN,NaN,NaN,NaN
3,2014,4,Changi,12,9.500000,7,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,5,Changi,15,10.333333,7,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
295,2019,48,Changi,52,52.000000,52,5.821429,22.35,28.267857,10.025,6.750000,48.333333
296,2019,49,Changi,34,34.000000,34,2.107143,8.95,28.071429,9.925,6.882143,50.666667
297,2019,50,Changi,26,26.000000,26,4.378571,17.50,27.975000,9.550,7.064286,55.333333
298,2019,51,Changi,33,33.000000,33,6.428571,28.35,27.650000,9.125,8.303571,50.666667


In [152]:
changi_df = pd.read_csv("Datasets/Model Training merged data/Changi Weather and Cases.csv")
psi_max_avg_df = pd.read_csv("Datasets/PSI Data/psi_max_avg.csv")

#columns = ['Year', 'Week', 'Max National', 'Avg National', 'Max South', 'Avg South', 'Max North', 'Avg North', 'Max East'
        #, 'Avg East', 'Max Central', 'Avg Central', 'Max West', 'Avg West']
columns = ['Year', 'Week', 'Max PSI East', 'Avg PSI East']
psi_max_avg_changi_df = pd.DataFrame(columns=columns) 

index = 0

for i in range (2014, 2020):
    psi_year_df = psi_max_avg_df[(psi_max_avg_df['Year'] == i)]
    changi_year_df = changi_df[changi_df['Year'] == i]
    for j in range(1,54):
        psi_week_df = psi_year_df[(psi_year_df['Week'] == j)]
        changi_week_df = changi_year_df[changi_year_df['Week'] == j]
        row = []
        if len(changi_week_df) == 0:
            continue
        if len(psi_week_df) == 0:
            row = [i, j, np.nan, np.nan]
        else:
            values = psi_week_df.values.tolist()[0]
            row = [i, j, values[8], values[9]]
            
        psi_max_avg_changi_df.loc[index] = row
        index += 1
        
del psi_max_avg_changi_df['Year']
del psi_max_avg_changi_df['Week']
result = pd.concat([changi_df, psi_max_avg_changi_df], axis=1).reindex(changi_df.index)
result.to_csv("Datasets/Model Training merged data/Changi Weather PSI and Cases.csv", index=False)

In [134]:
columns = ['Year', 'Week', 'weather station', 'Max cases with onset in last 2 weeks', 'Average cases with onset in last 2 weeks', 'Min cases with onset in last 2 weeks']

location_cases_khatib_df = pd.DataFrame(columns=columns)

weather_khatib_df = pd.read_csv("Datasets/Weather Data/Weekly Khatib.csv")

location_dengue_max_avg_df = pd.read_csv("Datasets/Dengue Location Cluster Data/location_dengue_max_min_avg_final.csv")

#for khatib
index = 0
for i in range (2014, 2020):
    location_year_df = location_dengue_max_avg_df[(location_dengue_max_avg_df['Year'] == i) & (location_dengue_max_avg_df['weather station'] == 'Khatib')]
    weather_year_df = weather_khatib_df[weather_khatib_df['Year'] == i]
    for j in range(1,54):
        location_week_df = location_year_df[(location_year_df['Week'] == j)]
        weather_week_df = weather_year_df[weather_year_df['Week'] == j]
        row = []
        if len(weather_week_df) == 0:
            continue
        if len(location_week_df) == 0:
            row = [i, j, 'Khatib', np.nan, np.nan, np.nan]
        else:
            row = location_week_df.values.tolist()[0]
        
        location_cases_khatib_df.loc[index] = row
        index += 1
        
del weather_khatib_df['Year']
del weather_khatib_df['Week']
result = pd.concat([location_cases_khatib_df, weather_khatib_df], axis=1).reindex(location_cases_khatib_df.index)
result.to_csv("Datasets/Model Training merged data/Khatib Weather and Cases.csv", index=False)



In [71]:
amk_weather_df = pd.read_csv("Datasets/Weather Data/Weekly Khatib.csv")


location_dengue_max_avg_df = pd.read_csv("Datasets/Dengue Location Cluster Data/location_dengue_max_min_avg_final.csv")
amk_location_dengue = location_dengue_max_avg_df[location_dengue_max_avg_df['weather station'] == 'Khatib']
amk_location_dengue.to_csv("Datasets/Model Training merged data/Khatib without weather.csv", index=False)
amk_location_dengue = pd.read_csv("Datasets/Model Training merged data/Khatib without weather.csv")

avg_daily_rainfall_total_col = pd.Series([], dtype='float64')
highest_60_min_rainfall_col = pd.Series([], dtype='float64')
mean_temperature_col = pd.Series([], dtype='float64')
range_temperature_col = pd.Series([], dtype='float64')
mean_wind_speed_col = pd.Series([], dtype='float64')
historical_cases_week_col = pd.Series([], dtype='int32')

for i in range(len(amk_location_dengue)):
    if amk_location_dengue['Year'][i] == 2014 and amk_location_dengue['Week'][i] <= 5:
        avg_daily_rainfall_total_col[i] = np.nan
        highest_60_min_rainfall_col[i] = np.nan
        mean_temperature_col[i] = np.nan
        range_temperature_col[i] = np.nan
        mean_wind_speed_col[i] = np.nan
        historical_cases_week_col[i] = np.nan
        continue
    if amk_location_dengue['Year'][i] == 2019 and amk_location_dengue['Week'][i] < 19 and amk_location_dengue['Week'][i] >= 14:
        avg_daily_rainfall_total_col[i] = np.nan
        highest_60_min_rainfall_col[i] = np.nan
        mean_temperature_col[i] = np.nan
        range_temperature_col[i] = np.nan
        mean_wind_speed_col[i] = np.nan
        historical_cases_week_col[i] = np.nan
        continue
    if amk_location_dengue['Year'][i] == 2015 and amk_location_dengue['Week'][i] < 14 and amk_location_dengue['Week'][i] >= 8:
        avg_daily_rainfall_total_col[i] = np.nan
        highest_60_min_rainfall_col[i] = np.nan
        mean_temperature_col[i] = np.nan
        range_temperature_col[i] = np.nan
        mean_wind_speed_col[i] = np.nan
        historical_cases_week_col[i] = np.nan
        continue
    
    #find number of weeks in previous year in case
    year = amk_location_dengue['Year'][i] - 1
    year_df = amk_weather_df[amk_weather_df['Year'] == year]
    year_week_df = year_df['Week']
    max_week = year_week_df.max()
    
    avg_daily_rainfall_total = 0
    highest_60_min_rainfall = 0
    mean_temperature = 0
    range_temperature = 0
    mean_wind_speed = 0
    historical_cases_week = 0
    

    week = amk_location_dengue['Week'][i]
    historical_week = week - 5
    for j in range(0, 4):
        if historical_week+j <= 0:
            weather_week = max_week + (historical_week+j)
            weather_year = year
        else:
            weather_week = historical_week + j
            weather_year = year + 1
        weather_df = amk_weather_df[(amk_weather_df['Year'] == weather_year) & (amk_weather_df['Week'] == weather_week)]
        location_df = amk_location_dengue[(amk_location_dengue['Year'] == weather_year) & (amk_location_dengue['Week'] == weather_week)]
        avg_daily_rainfall_total += weather_df.values.tolist()[0][4]
        highest_60_min_rainfall += weather_df.values.tolist()[0][6]
        mean_temperature += weather_df.values.tolist()[0][8]
        range_temperature += weather_df.values.tolist()[0][9] - weather_df.values.tolist()[0][10]
        mean_wind_speed += weather_df.values.tolist()[0][11]
        if j > 0:
            if len(location_df) == 0:
                historical_cases_week += 0
            else:
                historical_cases_week += location_df.values.tolist()[0][4]
                
    avg_daily_rainfall_total_col[i] = avg_daily_rainfall_total / 4
    highest_60_min_rainfall_col[i] = highest_60_min_rainfall / 4
    mean_temperature_col[i] = mean_temperature / 4
    range_temperature_col[i] = range_temperature / 4
    mean_wind_speed_col[i] = mean_wind_speed / 4
    historical_cases_week_col[i] = historical_cases_week / 3
    

    
amk_location_dengue.insert(6, 'avg_daily_rainfall_total', avg_daily_rainfall_total_col)
amk_location_dengue.insert(7, 'highest_60_min_rainfall', highest_60_min_rainfall_col)
amk_location_dengue.insert(8, 'mean_temperature', mean_temperature_col)
amk_location_dengue.insert(9, 'range_temperature', range_temperature_col)
amk_location_dengue.insert(10, 'mean_wind_speed', mean_wind_speed_col)
amk_location_dengue.insert(11, 'historical_cases_week', historical_cases_week_col)

amk_location_dengue.to_csv("Datasets/Model Training merged data/Khatib Aggregate.csv", index=False)
amk_location_dengue


,Year,Week,weather station,Max cases with onset in last 2 weeks,Average cases with onset in last 2 weeks,Min cases with onset in last 2 weeks,avg_daily_rainfall_total,highest_60_min_rainfall,mean_temperature,range_temperature,mean_wind_speed,historical_cases_week
0,2014,1,Khatib,20,20.000000,20,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,2,Khatib,35,25.000000,15,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,3,Khatib,41,37.500000,34,NaN,NaN,NaN,NaN,NaN,NaN
3,2014,4,Khatib,40,33.000000,26,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,5,Khatib,27,23.333333,19,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
295,2019,48,Khatib,96,96.000000,96,4.985714,16.375,26.732143,10.550,7.478571,76.333333
296,2019,49,Khatib,69,69.000000,69,4.821429,21.975,26.735714,10.550,7.200000,99.333333
297,2019,50,Khatib,75,75.000000,75,9.128571,33.925,26.846429,10.525,7.139286,107.000000
298,2019,51,Khatib,60,60.000000,60,9.907143,36.450,26.785714,10.425,7.592857,97.333333


In [153]:
khatib_df = pd.read_csv("Datasets/Model Training merged data/Khatib Weather and Cases.csv")
psi_max_avg_df = pd.read_csv("Datasets/PSI Data/psi_max_avg.csv")

#columns = ['Year', 'Week', 'Max National', 'Avg National', 'Max South', 'Avg South', 'Max North', 'Avg North', 'Max East'
        #, 'Avg East', 'Max Central', 'Avg Central', 'Max West', 'Avg West']
columns = ['Year', 'Week', 'Max PSI North', 'Avg PSI North']
psi_max_avg_khatib_df = pd.DataFrame(columns=columns) 

index = 0

for i in range (2014, 2020):
    psi_year_df = psi_max_avg_df[(psi_max_avg_df['Year'] == i)]
    khatib_year_df = khatib_df[khatib_df['Year'] == i]
    for j in range(1,54):
        psi_week_df = psi_year_df[(psi_year_df['Week'] == j)]
        khatib_week_df = khatib_year_df[khatib_year_df['Week'] == j]
        row = []
        if len(khatib_week_df) == 0:
            continue
        if len(psi_week_df) == 0:
            row = [i, j, np.nan, np.nan]
        else:
            values = psi_week_df.values.tolist()[0]
            row = [i, j, values[6], values[7]]
            
        psi_max_avg_khatib_df.loc[index] = row
        index += 1
        
del psi_max_avg_khatib_df['Year']
del psi_max_avg_khatib_df['Week']
result = pd.concat([khatib_df, psi_max_avg_khatib_df], axis=1).reindex(khatib_df.index)
result.to_csv("Datasets/Model Training merged data/Khatib Weather PSI and Cases.csv", index=False)

In [135]:
columns = ['Year', 'Week', 'weather station', 'Max cases with onset in last 2 weeks', 'Average cases with onset in last 2 weeks', 'Min cases with onset in last 2 weeks']

location_cases_mb_df = pd.DataFrame(columns=columns)

weather_mb_df = pd.read_csv("Datasets/Weather Data/Weekly Marina Barrage.csv")

location_dengue_max_avg_df = pd.read_csv("Datasets/Dengue Location Cluster Data/location_dengue_max_min_avg_final.csv")

#for mb
index = 0
for i in range (2014, 2020):
    location_year_df = location_dengue_max_avg_df[(location_dengue_max_avg_df['Year'] == i) & (location_dengue_max_avg_df['weather station'] == 'Marina Barrage')]
    weather_year_df = weather_mb_df[weather_mb_df['Year'] == i]
    for j in range(1,54):
        location_week_df = location_year_df[(location_year_df['Week'] == j)]
        weather_week_df = weather_year_df[weather_year_df['Week'] == j]
        row = []
        if len(weather_week_df) == 0:
            continue
        if len(location_week_df) == 0:
            row = [i, j, 'Marina Barrage', np.nan, np.nan, np.nan]
        else:
            row = location_week_df.values.tolist()[0]
        
        location_cases_mb_df.loc[index] = row
        index += 1
        
del weather_mb_df['Year']
del weather_mb_df['Week']
result = pd.concat([location_cases_mb_df, weather_mb_df], axis=1).reindex(location_cases_mb_df.index)
result.to_csv("Datasets/Model Training merged data/Marina Barrage Weather and Cases.csv", index=False)



In [72]:
amk_weather_df = pd.read_csv("Datasets/Weather Data/Weekly Marina Barrage.csv")


location_dengue_max_avg_df = pd.read_csv("Datasets/Dengue Location Cluster Data/location_dengue_max_min_avg_final.csv")
amk_location_dengue = location_dengue_max_avg_df[location_dengue_max_avg_df['weather station'] == 'Marina Barrage']
amk_location_dengue.to_csv("Datasets/Model Training merged data/Marina Barrage without weather.csv", index=False)
amk_location_dengue = pd.read_csv("Datasets/Model Training merged data/Marina Barrage without weather.csv")

avg_daily_rainfall_total_col = pd.Series([], dtype='float64')
highest_60_min_rainfall_col = pd.Series([], dtype='float64')
mean_temperature_col = pd.Series([], dtype='float64')
range_temperature_col = pd.Series([], dtype='float64')
mean_wind_speed_col = pd.Series([], dtype='float64')
historical_cases_week_col = pd.Series([], dtype='int32')

for i in range(len(amk_location_dengue)):
    if amk_location_dengue['Year'][i] == 2014 and amk_location_dengue['Week'][i] <= 5:
        avg_daily_rainfall_total_col[i] = np.nan
        highest_60_min_rainfall_col[i] = np.nan
        mean_temperature_col[i] = np.nan
        range_temperature_col[i] = np.nan
        mean_wind_speed_col[i] = np.nan
        historical_cases_week_col[i] = np.nan
        continue
    if amk_location_dengue['Year'][i] == 2019 and amk_location_dengue['Week'][i] < 19 and amk_location_dengue['Week'][i] >= 14:
        avg_daily_rainfall_total_col[i] = np.nan
        highest_60_min_rainfall_col[i] = np.nan
        mean_temperature_col[i] = np.nan
        range_temperature_col[i] = np.nan
        mean_wind_speed_col[i] = np.nan
        historical_cases_week_col[i] = np.nan
        continue
    
    #find number of weeks in previous year in case
    year = amk_location_dengue['Year'][i] - 1
    year_df = amk_weather_df[amk_weather_df['Year'] == year]
    year_week_df = year_df['Week']
    max_week = year_week_df.max()
    
    avg_daily_rainfall_total = 0
    highest_60_min_rainfall = 0
    mean_temperature = 0
    range_temperature = 0
    mean_wind_speed = 0
    historical_cases_week = 0
    

    week = amk_location_dengue['Week'][i]
    historical_week = week - 5
    for j in range(0, 4):
        if historical_week+j <= 0:
            weather_week = max_week + (historical_week+j)
            weather_year = year
        else:
            weather_week = historical_week + j
            weather_year = year + 1
        weather_df = amk_weather_df[(amk_weather_df['Year'] == weather_year) & (amk_weather_df['Week'] == weather_week)]
        location_df = amk_location_dengue[(amk_location_dengue['Year'] == weather_year) & (amk_location_dengue['Week'] == weather_week)]
        avg_daily_rainfall_total += weather_df.values.tolist()[0][4]
        highest_60_min_rainfall += weather_df.values.tolist()[0][6]
        mean_temperature += weather_df.values.tolist()[0][8]
        range_temperature += weather_df.values.tolist()[0][9] - weather_df.values.tolist()[0][10]
        mean_wind_speed += weather_df.values.tolist()[0][11]
        if j > 0:
            if len(location_df) == 0:
                historical_cases_week += 0
            else:
                historical_cases_week += location_df.values.tolist()[0][4]
                
    avg_daily_rainfall_total_col[i] = avg_daily_rainfall_total / 4
    highest_60_min_rainfall_col[i] = highest_60_min_rainfall / 4
    mean_temperature_col[i] = mean_temperature / 4
    range_temperature_col[i] = range_temperature / 4
    mean_wind_speed_col[i] = mean_wind_speed / 4
    historical_cases_week_col[i] = historical_cases_week / 3
    

    
amk_location_dengue.insert(6, 'avg_daily_rainfall_total', avg_daily_rainfall_total_col)
amk_location_dengue.insert(7, 'highest_60_min_rainfall', highest_60_min_rainfall_col)
amk_location_dengue.insert(8, 'mean_temperature', mean_temperature_col)
amk_location_dengue.insert(9, 'range_temperature', range_temperature_col)
amk_location_dengue.insert(10, 'mean_wind_speed', mean_wind_speed_col)
amk_location_dengue.insert(11, 'historical_cases_week', historical_cases_week_col)

amk_location_dengue.to_csv("Datasets/Model Training merged data/Marina Barrage Aggregate.csv", index=False)
amk_location_dengue

,Year,Week,weather station,Max cases with onset in last 2 weeks,Average cases with onset in last 2 weeks,Min cases with onset in last 2 weeks,avg_daily_rainfall_total,highest_60_min_rainfall,mean_temperature,range_temperature,mean_wind_speed,historical_cases_week
0,2014,1,Marina Barrage,21,21.0,21,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,2,Marina Barrage,43,31.5,20,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,3,Marina Barrage,40,37.5,35,NaN,NaN,NaN,NaN,NaN,NaN
3,2014,4,Marina Barrage,38,29.5,21,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,5,Marina Barrage,34,30.0,24,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
295,2019,48,Marina Barrage,18,18.0,18,2.948571,9.00,29.405833,9.800,5.344048,23.000000
296,2019,49,Marina Barrage,24,24.0,24,0.928571,3.95,29.124583,9.425,5.859048,24.000000
297,2019,50,Marina Barrage,29,29.0,29,1.913095,6.15,28.874583,9.900,6.330476,21.333333
298,2019,51,Marina Barrage,19,19.0,19,4.461429,16.85,28.459583,9.675,7.932143,21.333333


In [154]:
mb_df = pd.read_csv("Datasets/Model Training merged data/Marina Barrage Weather and Cases.csv")
psi_max_avg_df = pd.read_csv("Datasets/PSI Data/psi_max_avg.csv")

#columns = ['Year', 'Week', 'Max National', 'Avg National', 'Max South', 'Avg South', 'Max North', 'Avg North', 'Max East'
        #, 'Avg East', 'Max Central', 'Avg Central', 'Max West', 'Avg West']
columns = ['Year', 'Week', 'Max PSI South', 'Avg PSI South']
psi_max_avg_mb_df = pd.DataFrame(columns=columns) 

index = 0

for i in range (2014, 2020):
    psi_year_df = psi_max_avg_df[(psi_max_avg_df['Year'] == i)]
    mb_year_df = mb_df[mb_df['Year'] == i]
    for j in range(1,54):
        psi_week_df = psi_year_df[(psi_year_df['Week'] == j)]
        mb_week_df = mb_year_df[mb_year_df['Week'] == j]
        row = []
        if len(mb_week_df) == 0:
            continue
        if len(psi_week_df) == 0:
            row = [i, j, np.nan, np.nan]
        else:
            values = psi_week_df.values.tolist()[0]
            row = [i, j, values[4], values[5]]
            
        psi_max_avg_mb_df.loc[index] = row
        index += 1
        
del psi_max_avg_mb_df['Year']
del psi_max_avg_mb_df['Week']
result = pd.concat([mb_df, psi_max_avg_mb_df], axis=1).reindex(mb_df.index)
result.to_csv("Datasets/Model Training merged data/Marina Barrage Weather PSI and Cases.csv", index=False)

In [136]:
columns = ['Year', 'Week', 'weather station', 'Max cases with onset in last 2 weeks', 'Average cases with onset in last 2 weeks', 'Min cases with onset in last 2 weeks']

location_cases_ts_df = pd.DataFrame(columns=columns)

weather_ts_df = pd.read_csv("Datasets/Weather Data/Weekly Tuas South.csv")

location_dengue_max_avg_df = pd.read_csv("Datasets/Dengue Location Cluster Data/location_dengue_max_min_avg_final.csv")

#for ts
index = 0
for i in range (2014, 2020):
    location_year_df = location_dengue_max_avg_df[(location_dengue_max_avg_df['Year'] == i) & (location_dengue_max_avg_df['weather station'] == 'Tuas South')]
    weather_year_df = weather_ts_df[weather_ts_df['Year'] == i]
    for j in range(1,54):
        location_week_df = location_year_df[(location_year_df['Week'] == j)]
        weather_week_df = weather_year_df[weather_year_df['Week'] == j]
        row = []
        if len(weather_week_df) == 0:
            continue
        if len(location_week_df) == 0:
            row = [i, j, 'Tuas South', np.nan, np.nan, np.nan]
        else:
            row = location_week_df.values.tolist()[0]
        
        location_cases_ts_df.loc[index] = row
        index += 1
        
del weather_ts_df['Year']
del weather_ts_df['Week']
result = pd.concat([location_cases_ts_df, weather_ts_df], axis=1).reindex(location_cases_ts_df.index)
result.to_csv("Datasets/Model Training merged data/Tuas South Weather and Cases.csv", index=False)


In [73]:
amk_weather_df = pd.read_csv("Datasets/Weather Data/Weekly Tuas South.csv")


location_dengue_max_avg_df = pd.read_csv("Datasets/Dengue Location Cluster Data/location_dengue_max_min_avg_final.csv")
amk_location_dengue = location_dengue_max_avg_df[location_dengue_max_avg_df['weather station'] == 'Tuas South']
amk_location_dengue.to_csv("Datasets/Model Training merged data/Tuas South without weather.csv", index=False)
amk_location_dengue = pd.read_csv("Datasets/Model Training merged data/Tuas South without weather.csv")

avg_daily_rainfall_total_col = pd.Series([], dtype='float64')
highest_60_min_rainfall_col = pd.Series([], dtype='float64')
mean_temperature_col = pd.Series([], dtype='float64')
range_temperature_col = pd.Series([], dtype='float64')
mean_wind_speed_col = pd.Series([], dtype='float64')
historical_cases_week_col = pd.Series([], dtype='int32')

for i in range(len(amk_location_dengue)):
    if amk_location_dengue['Year'][i] == 2014 and amk_location_dengue['Week'][i] <= 5:
        avg_daily_rainfall_total_col[i] = np.nan
        highest_60_min_rainfall_col[i] = np.nan
        mean_temperature_col[i] = np.nan
        range_temperature_col[i] = np.nan
        mean_wind_speed_col[i] = np.nan
        historical_cases_week_col[i] = np.nan
        continue
    if amk_location_dengue['Year'][i] == 2019 and amk_location_dengue['Week'][i] < 19 and amk_location_dengue['Week'][i] >= 14:
        avg_daily_rainfall_total_col[i] = np.nan
        highest_60_min_rainfall_col[i] = np.nan
        mean_temperature_col[i] = np.nan
        range_temperature_col[i] = np.nan
        mean_wind_speed_col[i] = np.nan
        historical_cases_week_col[i] = np.nan
        continue
    
    #find number of weeks in previous year in case
    year = amk_location_dengue['Year'][i] - 1
    year_df = amk_weather_df[amk_weather_df['Year'] == year]
    year_week_df = year_df['Week']
    max_week = year_week_df.max()
    
    avg_daily_rainfall_total = 0
    highest_60_min_rainfall = 0
    mean_temperature = 0
    range_temperature = 0
    mean_wind_speed = 0
    historical_cases_week = 0
    

    week = amk_location_dengue['Week'][i]
    historical_week = week - 5
    for j in range(0, 4):
        if historical_week+j <= 0:
            weather_week = max_week + (historical_week+j)
            weather_year = year
        else:
            weather_week = historical_week + j
            weather_year = year + 1
        weather_df = amk_weather_df[(amk_weather_df['Year'] == weather_year) & (amk_weather_df['Week'] == weather_week)]
        location_df = amk_location_dengue[(amk_location_dengue['Year'] == weather_year) & (amk_location_dengue['Week'] == weather_week)]
        avg_daily_rainfall_total += weather_df.values.tolist()[0][4]
        highest_60_min_rainfall += weather_df.values.tolist()[0][6]
        mean_temperature += weather_df.values.tolist()[0][8]
        range_temperature += weather_df.values.tolist()[0][9] - weather_df.values.tolist()[0][10]
        mean_wind_speed += weather_df.values.tolist()[0][11]
        if j > 0:
            if len(location_df) == 0:
                historical_cases_week += 0
            else:
                historical_cases_week += location_df.values.tolist()[0][4]
                
    avg_daily_rainfall_total_col[i] = avg_daily_rainfall_total / 4
    highest_60_min_rainfall_col[i] = highest_60_min_rainfall / 4
    mean_temperature_col[i] = mean_temperature / 4
    range_temperature_col[i] = range_temperature / 4
    mean_wind_speed_col[i] = mean_wind_speed / 4
    historical_cases_week_col[i] = historical_cases_week / 3
    

    
amk_location_dengue.insert(6, 'avg_daily_rainfall_total', avg_daily_rainfall_total_col)
amk_location_dengue.insert(7, 'highest_60_min_rainfall', highest_60_min_rainfall_col)
amk_location_dengue.insert(8, 'mean_temperature', mean_temperature_col)
amk_location_dengue.insert(9, 'range_temperature', range_temperature_col)
amk_location_dengue.insert(10, 'mean_wind_speed', mean_wind_speed_col)
amk_location_dengue.insert(11, 'historical_cases_week', historical_cases_week_col)

amk_location_dengue.to_csv("Datasets/Model Training merged data/Tuas South Aggregate.csv", index=False)
amk_location_dengue

,Year,Week,weather station,Max cases with onset in last 2 weeks,Average cases with onset in last 2 weeks,Min cases with onset in last 2 weeks,avg_daily_rainfall_total,highest_60_min_rainfall,mean_temperature,range_temperature,mean_wind_speed,historical_cases_week
0,2014,1,Tuas South,7,7.0,7,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,2,Tuas South,34,22.0,10,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,3,Tuas South,26,26.0,26,NaN,NaN,NaN,NaN,NaN,NaN
3,2014,4,Tuas South,14,13.5,13,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,5,Tuas South,8,7.0,6,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
295,2019,48,Tuas South,40,40.0,40,7.814286,28.70,28.292857,8.700,6.228571,29.666667
296,2019,49,Tuas South,23,23.0,23,5.642857,26.50,28.207143,8.600,6.189286,41.333333
297,2019,50,Tuas South,7,7.0,7,6.585714,25.15,28.003571,8.650,5.664286,44.333333
298,2019,51,Tuas South,10,10.0,10,5.571429,18.55,27.703571,8.625,5.653571,36.666667


In [156]:
ts_df = pd.read_csv("Datasets/Model Training merged data/Tuas South Weather and Cases.csv")
psi_max_avg_df = pd.read_csv("Datasets/PSI Data/psi_max_avg.csv")

#columns = ['Year', 'Week', 'Max National', 'Avg National', 'Max South', 'Avg South', 'Max North', 'Avg North', 'Max East'
        #, 'Avg East', 'Max Central', 'Avg Central', 'Max West', 'Avg West']
columns = ['Year', 'Week', 'Max PSI West', 'Avg PSI West']
psi_max_avg_ts_df = pd.DataFrame(columns=columns) 

index = 0

for i in range (2014, 2020):
    psi_year_df = psi_max_avg_df[(psi_max_avg_df['Year'] == i)]
    ts_year_df = ts_df[ts_df['Year'] == i]
    for j in range(1,54):
        psi_week_df = psi_year_df[(psi_year_df['Week'] == j)]
        ts_week_df = ts_year_df[ts_year_df['Week'] == j]
        row = []
        if len(ts_week_df) == 0:
            continue
        if len(psi_week_df) == 0:
            row = [i, j, np.nan, np.nan]
        else:
            values = psi_week_df.values.tolist()[0]
            row = [i, j, values[12], values[13]]
            
        psi_max_avg_ts_df.loc[index] = row
        index += 1
        
del psi_max_avg_ts_df['Year']
del psi_max_avg_ts_df['Week']
result = pd.concat([ts_df, psi_max_avg_ts_df], axis=1).reindex(ts_df.index)
result.to_csv("Datasets/Model Training merged data/Tuas South Weather PSI and Cases.csv", index=False)

append the aggregates for each location

In [74]:
amk_df = pd.read_csv("Datasets/Model Training merged data/Ang Mo Kio Aggregate.csv")
changi_df = pd.read_csv("Datasets/Model Training merged data/Changi Aggregate.csv")
khatib_df = pd.read_csv("Datasets/Model Training merged data/Khatib Aggregate.csv")
mb_df = pd.read_csv("Datasets/Model Training merged data/Marina Barrage Aggregate.csv")
ts_df = pd.read_csv("Datasets/Model Training merged data/Tuas South Aggregate.csv")

aggregate_df = amk_df.append([changi_df, khatib_df, mb_df, ts_df])
aggregate_df.to_csv("Datasets/Model Training merged data/location_cluster_aggregate_2_weeks.csv")
aggregate_df

,Year,Week,weather station,Max cases with onset in last 2 weeks,Average cases with onset in last 2 weeks,Min cases with onset in last 2 weeks,avg_daily_rainfall_total,highest_60_min_rainfall,mean_temperature,range_temperature,mean_wind_speed,historical_cases_week
0,2014,1,Ang Mo Kio,87,87.000000,87,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,2,Ang Mo Kio,91,79.500000,68,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,3,Ang Mo Kio,95,91.500000,88,NaN,NaN,NaN,NaN,NaN,NaN
3,2014,4,Ang Mo Kio,69,69.000000,69,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,5,Ang Mo Kio,57,54.666667,50,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
295,2019,48,Tuas South,40,40.000000,40,7.814286,28.70,28.292857,8.700,6.228571,29.666667
296,2019,49,Tuas South,23,23.000000,23,5.642857,26.50,28.207143,8.600,6.189286,41.333333
297,2019,50,Tuas South,7,7.000000,7,6.585714,25.15,28.003571,8.650,5.664286,44.333333
298,2019,51,Tuas South,10,10.000000,10,5.571429,18.55,27.703571,8.625,5.653571,36.666667


4 weeks prediction for aggregate (location cluster)

In [75]:
amk_weather_df = pd.read_csv("Datasets/Weather Data/Weekly Tuas South.csv")


location_dengue_max_avg_df = pd.read_csv("Datasets/Dengue Location Cluster Data/location_dengue_max_min_avg_final.csv")
amk_location_dengue = location_dengue_max_avg_df[location_dengue_max_avg_df['weather station'] == 'Tuas South']
amk_location_dengue.to_csv("Datasets/Model Training merged data/Tuas South without weather.csv", index=False)
amk_location_dengue = pd.read_csv("Datasets/Model Training merged data/Tuas South without weather.csv")

avg_daily_rainfall_total_col = pd.Series([], dtype='float64')
highest_60_min_rainfall_col = pd.Series([], dtype='float64')
mean_temperature_col = pd.Series([], dtype='float64')
range_temperature_col = pd.Series([], dtype='float64')
mean_wind_speed_col = pd.Series([], dtype='float64')
historical_cases_week_col = pd.Series([], dtype='int32')

for i in range(len(amk_location_dengue)):
    if amk_location_dengue['Year'][i] == 2014 and amk_location_dengue['Week'][i] <= 7:
        avg_daily_rainfall_total_col[i] = np.nan
        highest_60_min_rainfall_col[i] = np.nan
        mean_temperature_col[i] = np.nan
        range_temperature_col[i] = np.nan
        mean_wind_speed_col[i] = np.nan
        historical_cases_week_col[i] = np.nan
        continue
    if amk_location_dengue['Year'][i] == 2019 and amk_location_dengue['Week'][i] < 21 and amk_location_dengue['Week'][i] >= 14:
        avg_daily_rainfall_total_col[i] = np.nan
        highest_60_min_rainfall_col[i] = np.nan
        mean_temperature_col[i] = np.nan
        range_temperature_col[i] = np.nan
        mean_wind_speed_col[i] = np.nan
        historical_cases_week_col[i] = np.nan
        continue
    
    #find number of weeks in previous year in case
    year = amk_location_dengue['Year'][i] - 1
    year_df = amk_weather_df[amk_weather_df['Year'] == year]
    year_week_df = year_df['Week']
    max_week = year_week_df.max()
    
    avg_daily_rainfall_total = 0
    highest_60_min_rainfall = 0
    mean_temperature = 0
    range_temperature = 0
    mean_wind_speed = 0
    historical_cases_week = 0
    

    week = amk_location_dengue['Week'][i]
    historical_week = week - 7
    for j in range(0, 4):
        if historical_week+j <= 0:
            weather_week = max_week + (historical_week+j)
            weather_year = year
        else:
            weather_week = historical_week + j
            weather_year = year + 1
        weather_df = amk_weather_df[(amk_weather_df['Year'] == weather_year) & (amk_weather_df['Week'] == weather_week)]
        location_df = amk_location_dengue[(amk_location_dengue['Year'] == weather_year) & (amk_location_dengue['Week'] == weather_week)]
        avg_daily_rainfall_total += weather_df.values.tolist()[0][4]
        highest_60_min_rainfall += weather_df.values.tolist()[0][6]
        mean_temperature += weather_df.values.tolist()[0][8]
        range_temperature += weather_df.values.tolist()[0][9] - weather_df.values.tolist()[0][10]
        mean_wind_speed += weather_df.values.tolist()[0][11]
        if j > 0:
            if len(location_df) == 0:
                historical_cases_week += 0
            else:
                historical_cases_week += location_df.values.tolist()[0][4]
                
    avg_daily_rainfall_total_col[i] = avg_daily_rainfall_total / 4
    highest_60_min_rainfall_col[i] = highest_60_min_rainfall / 4
    mean_temperature_col[i] = mean_temperature / 4
    range_temperature_col[i] = range_temperature / 4
    mean_wind_speed_col[i] = mean_wind_speed / 4
    historical_cases_week_col[i] = historical_cases_week / 3
    

    
amk_location_dengue.insert(6, 'avg_daily_rainfall_total', avg_daily_rainfall_total_col)
amk_location_dengue.insert(7, 'highest_60_min_rainfall', highest_60_min_rainfall_col)
amk_location_dengue.insert(8, 'mean_temperature', mean_temperature_col)
amk_location_dengue.insert(9, 'range_temperature', range_temperature_col)
amk_location_dengue.insert(10, 'mean_wind_speed', mean_wind_speed_col)
amk_location_dengue.insert(11, 'historical_cases_week', historical_cases_week_col)

amk_location_dengue.to_csv("Datasets/Model Training merged data/Tuas South Aggregate 4 week.csv", index=False)
amk_location_dengue

,Year,Week,weather station,Max cases with onset in last 2 weeks,Average cases with onset in last 2 weeks,Min cases with onset in last 2 weeks,avg_daily_rainfall_total,highest_60_min_rainfall,mean_temperature,range_temperature,mean_wind_speed,historical_cases_week
0,2014,1,Tuas South,7,7.0,7,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,2,Tuas South,34,22.0,10,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,3,Tuas South,26,26.0,26,NaN,NaN,NaN,NaN,NaN,NaN
3,2014,4,Tuas South,14,13.5,13,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,5,Tuas South,8,7.0,6,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
295,2019,48,Tuas South,40,40.0,40,8.507143,24.45,28.067857,8.65,5.400000,11.000000
296,2019,49,Tuas South,23,23.0,23,7.671429,27.55,28.310714,8.70,5.982143,17.666667
297,2019,50,Tuas South,7,7.0,7,7.814286,28.70,28.292857,8.70,6.228571,29.666667
298,2019,51,Tuas South,10,10.0,10,5.642857,26.50,28.207143,8.60,6.189286,41.333333


In [77]:
amk_weather_df = pd.read_csv("Datasets/Weather Data/Weekly Ang Mo Kio.csv")


location_dengue_max_avg_df = pd.read_csv("Datasets/Dengue Location Cluster Data/location_dengue_max_min_avg_final.csv")
amk_location_dengue = location_dengue_max_avg_df[location_dengue_max_avg_df['weather station'] == 'Ang Mo Kio']
amk_location_dengue.to_csv("Datasets/Model Training merged data/Ang Mo Kio without weather.csv", index=False)
amk_location_dengue = pd.read_csv("Datasets/Model Training merged data/Ang Mo Kio without weather.csv")

avg_daily_rainfall_total_col = pd.Series([], dtype='float64')
highest_60_min_rainfall_col = pd.Series([], dtype='float64')
mean_temperature_col = pd.Series([], dtype='float64')
range_temperature_col = pd.Series([], dtype='float64')
mean_wind_speed_col = pd.Series([], dtype='float64')
historical_cases_week_col = pd.Series([], dtype='int32')

for i in range(len(amk_location_dengue)):
    if amk_location_dengue['Year'][i] == 2014 and amk_location_dengue['Week'][i] <= 7:
        avg_daily_rainfall_total_col[i] = np.nan
        highest_60_min_rainfall_col[i] = np.nan
        mean_temperature_col[i] = np.nan
        range_temperature_col[i] = np.nan
        mean_wind_speed_col[i] = np.nan
        historical_cases_week_col[i] = np.nan
        continue
    if amk_location_dengue['Year'][i] == 2019 and amk_location_dengue['Week'][i] < 21 and amk_location_dengue['Week'][i] >= 14:
        avg_daily_rainfall_total_col[i] = np.nan
        highest_60_min_rainfall_col[i] = np.nan
        mean_temperature_col[i] = np.nan
        range_temperature_col[i] = np.nan
        mean_wind_speed_col[i] = np.nan
        historical_cases_week_col[i] = np.nan
        continue
    
    #find number of weeks in previous year in case
    year = amk_location_dengue['Year'][i] - 1
    year_df = amk_weather_df[amk_weather_df['Year'] == year]
    year_week_df = year_df['Week']
    max_week = year_week_df.max()
    
    avg_daily_rainfall_total = 0
    highest_60_min_rainfall = 0
    mean_temperature = 0
    range_temperature = 0
    mean_wind_speed = 0
    historical_cases_week = 0
    

    week = amk_location_dengue['Week'][i]
    historical_week = week - 7
    for j in range(0, 4):
        if historical_week+j <= 0:
            weather_week = max_week + (historical_week+j)
            weather_year = year
        else:
            weather_week = historical_week + j
            weather_year = year + 1
        weather_df = amk_weather_df[(amk_weather_df['Year'] == weather_year) & (amk_weather_df['Week'] == weather_week)]
        location_df = amk_location_dengue[(amk_location_dengue['Year'] == weather_year) & (amk_location_dengue['Week'] == weather_week)]
        avg_daily_rainfall_total += weather_df.values.tolist()[0][4]
        highest_60_min_rainfall += weather_df.values.tolist()[0][6]
        mean_temperature += weather_df.values.tolist()[0][8]
        range_temperature += weather_df.values.tolist()[0][9] - weather_df.values.tolist()[0][10]
        mean_wind_speed += weather_df.values.tolist()[0][11]
        if j > 0:
            if len(location_df) == 0:
                historical_cases_week += 0
            else:
                historical_cases_week += location_df.values.tolist()[0][4]
                
    avg_daily_rainfall_total_col[i] = avg_daily_rainfall_total / 4
    highest_60_min_rainfall_col[i] = highest_60_min_rainfall / 4
    mean_temperature_col[i] = mean_temperature / 4
    range_temperature_col[i] = range_temperature / 4
    mean_wind_speed_col[i] = mean_wind_speed / 4
    historical_cases_week_col[i] = historical_cases_week / 3
    

    
amk_location_dengue.insert(6, 'avg_daily_rainfall_total', avg_daily_rainfall_total_col)
amk_location_dengue.insert(7, 'highest_60_min_rainfall', highest_60_min_rainfall_col)
amk_location_dengue.insert(8, 'mean_temperature', mean_temperature_col)
amk_location_dengue.insert(9, 'range_temperature', range_temperature_col)
amk_location_dengue.insert(10, 'mean_wind_speed', mean_wind_speed_col)
amk_location_dengue.insert(11, 'historical_cases_week', historical_cases_week_col)

amk_location_dengue.to_csv("Datasets/Model Training merged data/Ang Mo Kio Aggregate 4 week.csv", index=False)
amk_location_dengue

,Year,Week,weather station,Max cases with onset in last 2 weeks,Average cases with onset in last 2 weeks,Min cases with onset in last 2 weeks,avg_daily_rainfall_total,highest_60_min_rainfall,mean_temperature,range_temperature,mean_wind_speed,historical_cases_week
0,2014,1,Ang Mo Kio,87,87.000000,87,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,2,Ang Mo Kio,91,79.500000,68,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,3,Ang Mo Kio,95,91.500000,88,NaN,NaN,NaN,NaN,NaN,NaN
3,2014,4,Ang Mo Kio,69,69.000000,69,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,5,Ang Mo Kio,57,54.666667,50,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
295,2019,48,Ang Mo Kio,143,143.000000,143,6.858571,19.05,27.894286,10.300,11.385714,102.333333
296,2019,49,Ang Mo Kio,146,146.000000,146,7.994286,26.90,28.115714,10.525,10.910714,125.333333
297,2019,50,Ang Mo Kio,111,111.000000,111,7.865714,25.85,28.008571,10.575,10.707143,141.000000
298,2019,51,Ang Mo Kio,93,93.000000,93,7.028571,25.85,27.857143,10.775,11.439286,149.333333


In [78]:
amk_weather_df = pd.read_csv("Datasets/Weather Data/Weekly Changi.csv")


location_dengue_max_avg_df = pd.read_csv("Datasets/Dengue Location Cluster Data/location_dengue_max_min_avg_final.csv")
amk_location_dengue = location_dengue_max_avg_df[location_dengue_max_avg_df['weather station'] == 'Changi']
amk_location_dengue.to_csv("Datasets/Model Training merged data/Changi without weather.csv", index=False)
amk_location_dengue = pd.read_csv("Datasets/Model Training merged data/Changi without weather.csv")

avg_daily_rainfall_total_col = pd.Series([], dtype='float64')
highest_60_min_rainfall_col = pd.Series([], dtype='float64')
mean_temperature_col = pd.Series([], dtype='float64')
range_temperature_col = pd.Series([], dtype='float64')
mean_wind_speed_col = pd.Series([], dtype='float64')
historical_cases_week_col = pd.Series([], dtype='int32')

for i in range(len(amk_location_dengue)):
    if amk_location_dengue['Year'][i] == 2014 and amk_location_dengue['Week'][i] <= 7:
        avg_daily_rainfall_total_col[i] = np.nan
        highest_60_min_rainfall_col[i] = np.nan
        mean_temperature_col[i] = np.nan
        range_temperature_col[i] = np.nan
        mean_wind_speed_col[i] = np.nan
        historical_cases_week_col[i] = np.nan
        continue
    if amk_location_dengue['Year'][i] == 2019 and amk_location_dengue['Week'][i] < 21 and amk_location_dengue['Week'][i] >= 14:
        avg_daily_rainfall_total_col[i] = np.nan
        highest_60_min_rainfall_col[i] = np.nan
        mean_temperature_col[i] = np.nan
        range_temperature_col[i] = np.nan
        mean_wind_speed_col[i] = np.nan
        historical_cases_week_col[i] = np.nan
        continue
    
    #find number of weeks in previous year in case
    year = amk_location_dengue['Year'][i] - 1
    year_df = amk_weather_df[amk_weather_df['Year'] == year]
    year_week_df = year_df['Week']
    max_week = year_week_df.max()
    
    avg_daily_rainfall_total = 0
    highest_60_min_rainfall = 0
    mean_temperature = 0
    range_temperature = 0
    mean_wind_speed = 0
    historical_cases_week = 0
    

    week = amk_location_dengue['Week'][i]
    historical_week = week - 7
    for j in range(0, 4):
        if historical_week+j <= 0:
            weather_week = max_week + (historical_week+j)
            weather_year = year
        else:
            weather_week = historical_week + j
            weather_year = year + 1
        weather_df = amk_weather_df[(amk_weather_df['Year'] == weather_year) & (amk_weather_df['Week'] == weather_week)]
        location_df = amk_location_dengue[(amk_location_dengue['Year'] == weather_year) & (amk_location_dengue['Week'] == weather_week)]
        avg_daily_rainfall_total += weather_df.values.tolist()[0][4]
        highest_60_min_rainfall += weather_df.values.tolist()[0][6]
        mean_temperature += weather_df.values.tolist()[0][8]
        range_temperature += weather_df.values.tolist()[0][9] - weather_df.values.tolist()[0][10]
        mean_wind_speed += weather_df.values.tolist()[0][11]
        if j > 0:
            if len(location_df) == 0:
                historical_cases_week += 0
            else:
                historical_cases_week += location_df.values.tolist()[0][4]
                
    avg_daily_rainfall_total_col[i] = avg_daily_rainfall_total / 4
    highest_60_min_rainfall_col[i] = highest_60_min_rainfall / 4
    mean_temperature_col[i] = mean_temperature / 4
    range_temperature_col[i] = range_temperature / 4
    mean_wind_speed_col[i] = mean_wind_speed / 4
    historical_cases_week_col[i] = historical_cases_week / 3
    

    
amk_location_dengue.insert(6, 'avg_daily_rainfall_total', avg_daily_rainfall_total_col)
amk_location_dengue.insert(7, 'highest_60_min_rainfall', highest_60_min_rainfall_col)
amk_location_dengue.insert(8, 'mean_temperature', mean_temperature_col)
amk_location_dengue.insert(9, 'range_temperature', range_temperature_col)
amk_location_dengue.insert(10, 'mean_wind_speed', mean_wind_speed_col)
amk_location_dengue.insert(11, 'historical_cases_week', historical_cases_week_col)

amk_location_dengue.to_csv("Datasets/Model Training merged data/Changi Aggregate 4 week.csv", index=False)
amk_location_dengue

,Year,Week,weather station,Max cases with onset in last 2 weeks,Average cases with onset in last 2 weeks,Min cases with onset in last 2 weeks,avg_daily_rainfall_total,highest_60_min_rainfall,mean_temperature,range_temperature,mean_wind_speed,historical_cases_week
0,2014,1,Changi,20,20.000000,20,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,2,Changi,21,17.500000,14,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,3,Changi,11,10.000000,9,NaN,NaN,NaN,NaN,NaN,NaN
3,2014,4,Changi,12,9.500000,7,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,5,Changi,15,10.333333,7,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
295,2019,48,Changi,52,52.000000,52,6.321429,23.95,28.267857,9.825,7.407143,58.666667
296,2019,49,Changi,34,34.000000,34,6.064286,22.75,28.378571,9.700,7.185714,55.333333
297,2019,50,Changi,26,26.000000,26,5.821429,22.35,28.267857,10.025,6.750000,48.333333
298,2019,51,Changi,33,33.000000,33,2.107143,8.95,28.071429,9.925,6.882143,50.666667


In [79]:
amk_weather_df = pd.read_csv("Datasets/Weather Data/Weekly Khatib.csv")


location_dengue_max_avg_df = pd.read_csv("Datasets/Dengue Location Cluster Data/location_dengue_max_min_avg_final.csv")
amk_location_dengue = location_dengue_max_avg_df[location_dengue_max_avg_df['weather station'] == 'Khatib']
amk_location_dengue.to_csv("Datasets/Model Training merged data/Khatib without weather.csv", index=False)
amk_location_dengue = pd.read_csv("Datasets/Model Training merged data/Khatib without weather.csv")

avg_daily_rainfall_total_col = pd.Series([], dtype='float64')
highest_60_min_rainfall_col = pd.Series([], dtype='float64')
mean_temperature_col = pd.Series([], dtype='float64')
range_temperature_col = pd.Series([], dtype='float64')
mean_wind_speed_col = pd.Series([], dtype='float64')
historical_cases_week_col = pd.Series([], dtype='int32')

for i in range(len(amk_location_dengue)):
    if amk_location_dengue['Year'][i] == 2014 and amk_location_dengue['Week'][i] <= 7:
        avg_daily_rainfall_total_col[i] = np.nan
        highest_60_min_rainfall_col[i] = np.nan
        mean_temperature_col[i] = np.nan
        range_temperature_col[i] = np.nan
        mean_wind_speed_col[i] = np.nan
        historical_cases_week_col[i] = np.nan
        continue
    if amk_location_dengue['Year'][i] == 2019 and amk_location_dengue['Week'][i] < 21 and amk_location_dengue['Week'][i] >= 14:
        avg_daily_rainfall_total_col[i] = np.nan
        highest_60_min_rainfall_col[i] = np.nan
        mean_temperature_col[i] = np.nan
        range_temperature_col[i] = np.nan
        mean_wind_speed_col[i] = np.nan
        historical_cases_week_col[i] = np.nan
        continue
    
    if amk_location_dengue['Year'][i] == 2015 and amk_location_dengue['Week'][i] < 16 and amk_location_dengue['Week'][i] >= 10:
        avg_daily_rainfall_total_col[i] = np.nan
        highest_60_min_rainfall_col[i] = np.nan
        mean_temperature_col[i] = np.nan
        range_temperature_col[i] = np.nan
        mean_wind_speed_col[i] = np.nan
        historical_cases_week_col[i] = np.nan
        continue
    
    #find number of weeks in previous year in case
    year = amk_location_dengue['Year'][i] - 1
    year_df = amk_weather_df[amk_weather_df['Year'] == year]
    year_week_df = year_df['Week']
    max_week = year_week_df.max()
    
    avg_daily_rainfall_total = 0
    highest_60_min_rainfall = 0
    mean_temperature = 0
    range_temperature = 0
    mean_wind_speed = 0
    historical_cases_week = 0
    

    week = amk_location_dengue['Week'][i]
    historical_week = week - 7
    for j in range(0, 4):
        if historical_week+j <= 0:
            weather_week = max_week + (historical_week+j)
            weather_year = year
        else:
            weather_week = historical_week + j
            weather_year = year + 1
        weather_df = amk_weather_df[(amk_weather_df['Year'] == weather_year) & (amk_weather_df['Week'] == weather_week)]
        location_df = amk_location_dengue[(amk_location_dengue['Year'] == weather_year) & (amk_location_dengue['Week'] == weather_week)]
        avg_daily_rainfall_total += weather_df.values.tolist()[0][4]
        highest_60_min_rainfall += weather_df.values.tolist()[0][6]
        mean_temperature += weather_df.values.tolist()[0][8]
        range_temperature += weather_df.values.tolist()[0][9] - weather_df.values.tolist()[0][10]
        mean_wind_speed += weather_df.values.tolist()[0][11]
        if j > 0:
            if len(location_df) == 0:
                historical_cases_week += 0
            else:
                historical_cases_week += location_df.values.tolist()[0][4]
                
    avg_daily_rainfall_total_col[i] = avg_daily_rainfall_total / 4
    highest_60_min_rainfall_col[i] = highest_60_min_rainfall / 4
    mean_temperature_col[i] = mean_temperature / 4
    range_temperature_col[i] = range_temperature / 4
    mean_wind_speed_col[i] = mean_wind_speed / 4
    historical_cases_week_col[i] = historical_cases_week / 3
    

    
amk_location_dengue.insert(6, 'avg_daily_rainfall_total', avg_daily_rainfall_total_col)
amk_location_dengue.insert(7, 'highest_60_min_rainfall', highest_60_min_rainfall_col)
amk_location_dengue.insert(8, 'mean_temperature', mean_temperature_col)
amk_location_dengue.insert(9, 'range_temperature', range_temperature_col)
amk_location_dengue.insert(10, 'mean_wind_speed', mean_wind_speed_col)
amk_location_dengue.insert(11, 'historical_cases_week', historical_cases_week_col)

amk_location_dengue.to_csv("Datasets/Model Training merged data/Khatib Aggregate 4 week.csv", index=False)
amk_location_dengue

,Year,Week,weather station,Max cases with onset in last 2 weeks,Average cases with onset in last 2 weeks,Min cases with onset in last 2 weeks,avg_daily_rainfall_total,highest_60_min_rainfall,mean_temperature,range_temperature,mean_wind_speed,historical_cases_week
0,2014,1,Khatib,20,20.000000,20,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,2,Khatib,35,25.000000,15,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,3,Khatib,41,37.500000,34,NaN,NaN,NaN,NaN,NaN,NaN
3,2014,4,Khatib,40,33.000000,26,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,5,Khatib,27,23.333333,19,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
295,2019,48,Khatib,96,96.000000,96,6.864286,22.125,26.400000,10.525,8.246429,42.000000
296,2019,49,Khatib,69,69.000000,69,5.132143,15.250,26.757143,10.500,7.867857,53.666667
297,2019,50,Khatib,75,75.000000,75,4.985714,16.375,26.732143,10.550,7.478571,76.333333
298,2019,51,Khatib,60,60.000000,60,4.821429,21.975,26.735714,10.550,7.200000,99.333333


In [80]:
amk_weather_df = pd.read_csv("Datasets/Weather Data/Weekly Marina Barrage.csv")


location_dengue_max_avg_df = pd.read_csv("Datasets/Dengue Location Cluster Data/location_dengue_max_min_avg_final.csv")
amk_location_dengue = location_dengue_max_avg_df[location_dengue_max_avg_df['weather station'] == 'Marina Barrage']
amk_location_dengue.to_csv("Datasets/Model Training merged data/Marina Barrage without weather.csv", index=False)
amk_location_dengue = pd.read_csv("Datasets/Model Training merged data/Marina Barrage without weather.csv")

avg_daily_rainfall_total_col = pd.Series([], dtype='float64')
highest_60_min_rainfall_col = pd.Series([], dtype='float64')
mean_temperature_col = pd.Series([], dtype='float64')
range_temperature_col = pd.Series([], dtype='float64')
mean_wind_speed_col = pd.Series([], dtype='float64')
historical_cases_week_col = pd.Series([], dtype='int32')

for i in range(len(amk_location_dengue)):
    if amk_location_dengue['Year'][i] == 2014 and amk_location_dengue['Week'][i] <= 7:
        avg_daily_rainfall_total_col[i] = np.nan
        highest_60_min_rainfall_col[i] = np.nan
        mean_temperature_col[i] = np.nan
        range_temperature_col[i] = np.nan
        mean_wind_speed_col[i] = np.nan
        historical_cases_week_col[i] = np.nan
        continue
    if amk_location_dengue['Year'][i] == 2019 and amk_location_dengue['Week'][i] < 21 and amk_location_dengue['Week'][i] >= 14:
        avg_daily_rainfall_total_col[i] = np.nan
        highest_60_min_rainfall_col[i] = np.nan
        mean_temperature_col[i] = np.nan
        range_temperature_col[i] = np.nan
        mean_wind_speed_col[i] = np.nan
        historical_cases_week_col[i] = np.nan
        continue
    
    #find number of weeks in previous year in case
    year = amk_location_dengue['Year'][i] - 1
    year_df = amk_weather_df[amk_weather_df['Year'] == year]
    year_week_df = year_df['Week']
    max_week = year_week_df.max()
    
    avg_daily_rainfall_total = 0
    highest_60_min_rainfall = 0
    mean_temperature = 0
    range_temperature = 0
    mean_wind_speed = 0
    historical_cases_week = 0
    

    week = amk_location_dengue['Week'][i]
    historical_week = week - 7
    for j in range(0, 4):
        if historical_week+j <= 0:
            weather_week = max_week + (historical_week+j)
            weather_year = year
        else:
            weather_week = historical_week + j
            weather_year = year + 1
        weather_df = amk_weather_df[(amk_weather_df['Year'] == weather_year) & (amk_weather_df['Week'] == weather_week)]
        location_df = amk_location_dengue[(amk_location_dengue['Year'] == weather_year) & (amk_location_dengue['Week'] == weather_week)]
        avg_daily_rainfall_total += weather_df.values.tolist()[0][4]
        highest_60_min_rainfall += weather_df.values.tolist()[0][6]
        mean_temperature += weather_df.values.tolist()[0][8]
        range_temperature += weather_df.values.tolist()[0][9] - weather_df.values.tolist()[0][10]
        mean_wind_speed += weather_df.values.tolist()[0][11]
        if j > 0:
            if len(location_df) == 0:
                historical_cases_week += 0
            else:
                historical_cases_week += location_df.values.tolist()[0][4]
                
    avg_daily_rainfall_total_col[i] = avg_daily_rainfall_total / 4
    highest_60_min_rainfall_col[i] = highest_60_min_rainfall / 4
    mean_temperature_col[i] = mean_temperature / 4
    range_temperature_col[i] = range_temperature / 4
    mean_wind_speed_col[i] = mean_wind_speed / 4
    historical_cases_week_col[i] = historical_cases_week / 3
    

    
amk_location_dengue.insert(6, 'avg_daily_rainfall_total', avg_daily_rainfall_total_col)
amk_location_dengue.insert(7, 'highest_60_min_rainfall', highest_60_min_rainfall_col)
amk_location_dengue.insert(8, 'mean_temperature', mean_temperature_col)
amk_location_dengue.insert(9, 'range_temperature', range_temperature_col)
amk_location_dengue.insert(10, 'mean_wind_speed', mean_wind_speed_col)
amk_location_dengue.insert(11, 'historical_cases_week', historical_cases_week_col)

amk_location_dengue.to_csv("Datasets/Model Training merged data/Marina Barrage Aggregate 4 week.csv", index=False)
amk_location_dengue

,Year,Week,weather station,Max cases with onset in last 2 weeks,Average cases with onset in last 2 weeks,Min cases with onset in last 2 weeks,avg_daily_rainfall_total,highest_60_min_rainfall,mean_temperature,range_temperature,mean_wind_speed,historical_cases_week
0,2014,1,Marina Barrage,21,21.0,21,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,2,Marina Barrage,43,31.5,20,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,3,Marina Barrage,40,37.5,35,NaN,NaN,NaN,NaN,NaN,NaN
3,2014,4,Marina Barrage,38,29.5,21,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,5,Marina Barrage,34,30.0,24,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
295,2019,48,Marina Barrage,18,18.0,18,2.735476,7.95,29.440833,8.875,4.835714,18.000000
296,2019,49,Marina Barrage,24,24.0,24,2.727143,7.85,29.530833,9.425,5.165476,19.000000
297,2019,50,Marina Barrage,29,29.0,29,2.948571,9.00,29.405833,9.800,5.344048,23.000000
298,2019,51,Marina Barrage,19,19.0,19,0.928571,3.95,29.124583,9.425,5.859048,24.000000


In [81]:
amk_df = pd.read_csv("Datasets/Model Training merged data/Ang Mo Kio Aggregate 4 week.csv")
changi_df = pd.read_csv("Datasets/Model Training merged data/Changi Aggregate 4 week.csv")
khatib_df = pd.read_csv("Datasets/Model Training merged data/Khatib Aggregate 4 week.csv")
mb_df = pd.read_csv("Datasets/Model Training merged data/Marina Barrage Aggregate 4 week.csv")
ts_df = pd.read_csv("Datasets/Model Training merged data/Tuas South Aggregate 4 week.csv")

aggregate_df = amk_df.append([changi_df, khatib_df, mb_df, ts_df])
aggregate_df.to_csv("Datasets/Model Training merged data/location_cluster_aggregate_4_weeks.csv")
aggregate_df

,Year,Week,weather station,Max cases with onset in last 2 weeks,Average cases with onset in last 2 weeks,Min cases with onset in last 2 weeks,avg_daily_rainfall_total,highest_60_min_rainfall,mean_temperature,range_temperature,mean_wind_speed,historical_cases_week
0,2014,1,Ang Mo Kio,87,87.000000,87,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,2,Ang Mo Kio,91,79.500000,68,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,3,Ang Mo Kio,95,91.500000,88,NaN,NaN,NaN,NaN,NaN,NaN
3,2014,4,Ang Mo Kio,69,69.000000,69,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,5,Ang Mo Kio,57,54.666667,50,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
295,2019,48,Tuas South,40,40.000000,40,8.507143,24.45,28.067857,8.65,5.400000,11.000000
296,2019,49,Tuas South,23,23.000000,23,7.671429,27.55,28.310714,8.70,5.982143,17.666667
297,2019,50,Tuas South,7,7.000000,7,7.814286,28.70,28.292857,8.70,6.228571,29.666667
298,2019,51,Tuas South,10,10.000000,10,5.642857,26.50,28.207143,8.60,6.189286,41.333333


Next, we can visualize the data.

In [ ]:
# Copy the dataframe, not to corrupt the original
vis_df = df.copy()
x_data = range(0, vis_df.shape[0])

# columns to visualize
columns = ['cases_per_capita', 'mean_sunshine_hrs', 'temp_mean_daily_max']

# plot line graphs
for column in columns:
    f = plt.figure()
    ax = f.add_subplot()
    ax.plot(x_data, vis_df[column], label=column)
    ax.legend()

In [ ]:
# fix random seed for reproducibility
np.random.seed(7)

# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)



Here we extract the min, max and avg of same week
First convert Ang Mo Kio station.

In [121]:
# Read from CSV
raw_AngMoKio_df = pd.read_csv(
    "Datasets/Weather Data/Ang Mo Kio.csv")
raw_Changi_df = pd.read_csv(
    "Datasets/Weather Data/Changi.csv")
raw_Khatib_df = pd.read_csv(
    "Datasets/Weather Data/Khatib.csv")
raw_MarinaBarrage_df = pd.read_csv(
    "Datasets/Weather Data/Marina Barrage.csv")
raw_TuasSouth_df = pd.read_csv(
    "Datasets/Weather Data/Tuas South.csv")

columns = ['Year', 'Week', 'Max Daily Rainfall Total (mm)', 'Min Daily Rainfall Total (mm)',
               'Average Daily Rainfall Total (mm)', 'Highest 30 Min Rainfall (mm)', 'Highest 60 Min Rainfall (mm)',
                  'Highest 120 Min Rainfall (mm)', 'Mean Temperature (°C)', 'Maximum Temperature (°C)',
                      'Minimum Temperature (°C)', 'Mean Wind Speed (km/h)', 'Max Wind Speed (km/h)']

del raw_AngMoKio_df['Station']
AngMoKio_df = pd.DataFrame(columns=columns)

def isfloat(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

index = 0
for i in range(2014, 2020):
    for j in range(1, 54):
        temp_df = raw_AngMoKio_df[(raw_AngMoKio_df['Year'] == i) & (raw_AngMoKio_df['Week'] == j)]
        if len(temp_df) == 0:
            continue
            
        rainSerie = temp_df['Daily Rainfall Total (mm)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        d30RainSerie = temp_df['Highest 30 Min Rainfall (mm)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        d60RainSerie = temp_df['Highest 60 Min Rainfall (mm)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        d120RainSerie = temp_df['Highest 120 Min Rainfall (mm)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        meanTempSerie = temp_df['Mean Temperature (°C)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        maxTempSerie = temp_df['Maximum Temperature (°C)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        minTempSerie = temp_df['Minimum Temperature (°C)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        meanWindSerie = temp_df['Mean Wind Speed (km/h)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        maxWindSerie = temp_df['Max Wind Speed (km/h)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        maxRain = rainSerie.max()
        minRain = rainSerie.min()
        avgRain = rainSerie.sum() / len(rainSerie.dropna())
        max30Rain = d30RainSerie.max()
        max60Rain = d60RainSerie.max()
        max120Rain = d120RainSerie.max()
        avgTemp = meanTempSerie.sum() / len(meanTempSerie.dropna())
        maxTemp = maxTempSerie.max()
        minTemp = minTempSerie.min()
        avgWind = meanWindSerie.sum() / len(meanWindSerie.dropna())
        maxWind = maxWindSerie.max()
        AngMoKio_df.loc[index] = [i, j, maxRain, minRain, avgRain, max30Rain, max60Rain, max120Rain, avgTemp,
                                         maxTemp, minTemp, avgWind, maxWind]
        index = index + 1

AngMoKio_df


,Year,Week,Max Daily Rainfall Total (mm),Min Daily Rainfall Total (mm),Average Daily Rainfall Total (mm),Highest 30 Min Rainfall (mm),Highest 60 Min Rainfall (mm),Highest 120 Min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h)
0,2014.0,1.0,8.8,0.0,3.480000,8.6,8.8,8.8,26.160000,30.9,23.3,6.380000,32.0
1,2014.0,2.0,40.0,0.0,7.028571,8.4,9.4,13.8,26.157143,31.8,23.3,5.657143,35.6
2,2014.0,3.0,0.0,0.0,0.000000,0.0,0.0,0.0,25.742857,29.4,23.3,9.328571,41.4
3,2014.0,4.0,0.0,0.0,0.000000,0.0,0.0,0.0,25.342857,29.3,22.5,9.157143,38.9
4,2014.0,5.0,0.0,0.0,0.000000,0.0,0.0,0.0,25.728571,31.6,22.2,8.900000,40.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,2019.0,48.0,31.8,0.0,11.700000,18.0,26.4,31.8,27.833333,34.5,24.9,13.700000,38.9
309,2019.0,49.0,0.2,0.0,0.050000,0.2,0.2,0.2,27.050000,31.2,25.1,15.500000,50.4
310,2019.0,50.0,46.2,0.6,27.714286,13.2,17.4,25.6,24.671429,31.0,22.1,13.485714,45.0
311,2019.0,51.0,7.8,0.0,2.371429,7.4,7.4,7.4,26.214286,30.9,22.8,14.400000,38.5


Convert Changi station

In [122]:
del raw_Changi_df['Station']
Changi_df = pd.DataFrame(columns=columns)        
        
index = 0
for i in range(2014, 2020):
    for j in range(1, 54):
        temp_df = raw_Changi_df[(raw_Changi_df['Year'] == i) & (raw_Changi_df['Week'] == j)]
        if len(temp_df) == 0:
            continue
            
        rainSerie = temp_df['Daily Rainfall Total (mm)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        d30RainSerie = temp_df['Highest 30 Min Rainfall (mm)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        d60RainSerie = temp_df['Highest 60 Min Rainfall (mm)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        d120RainSerie = temp_df['Highest 120 Min Rainfall (mm)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        meanTempSerie = temp_df['Mean Temperature (°C)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        maxTempSerie = temp_df['Maximum Temperature (°C)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        minTempSerie = temp_df['Minimum Temperature (°C)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        meanWindSerie = temp_df['Mean Wind Speed (km/h)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        maxWindSerie = temp_df['Max Wind Speed (km/h)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        maxRain = rainSerie.max()
        minRain = rainSerie.min()
        avgRain = rainSerie.sum() / len(rainSerie.dropna())
        max30Rain = d30RainSerie.max()
        max60Rain = d60RainSerie.max()
        max120Rain = d120RainSerie.max()
        avgTemp = meanTempSerie.sum() / len(meanTempSerie.dropna())
        maxTemp = maxTempSerie.max()
        minTemp = minTempSerie.min()
        avgWind = meanWindSerie.sum() / len(meanWindSerie.dropna())
        maxWind = maxWindSerie.max()
        Changi_df.loc[index] = [i, j, maxRain, minRain, avgRain, max30Rain, max60Rain, max120Rain, avgTemp,
                                         maxTemp, minTemp, avgWind, maxWind]
        index = index + 1
Changi_df

,Year,Week,Max Daily Rainfall Total (mm),Min Daily Rainfall Total (mm),Average Daily Rainfall Total (mm),Highest 30 Min Rainfall (mm),Highest 60 Min Rainfall (mm),Highest 120 Min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h)
0,2014.0,1.0,18.4,0.0,3.680000,8.6,10.8,15.4,26.620000,31.1,23.3,8.960000,36.0
1,2014.0,2.0,31.2,0.0,8.142857,18.2,18.2,18.2,26.371429,31.8,23.7,7.514286,34.9
2,2014.0,3.0,0.0,0.0,0.000000,0.0,0.0,0.0,26.242857,30.3,23.5,13.214286,40.0
3,2014.0,4.0,0.0,0.0,0.000000,0.0,0.0,0.0,25.700000,29.4,22.7,12.285714,41.4
4,2014.0,5.0,0.0,0.0,0.000000,0.0,0.0,0.0,26.057143,31.6,22.4,13.671429,42.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,2019.0,48.0,73.4,0.0,13.371429,36.0,48.8,55.6,27.714286,33.0,24.0,7.357143,34.9
309,2019.0,49.0,59.6,0.0,8.971429,27.4,47.2,56.8,27.471429,31.9,23.9,12.000000,42.8
310,2019.0,50.0,74.2,3.4,43.400000,20.6,26.2,36.8,24.914286,31.1,22.7,7.471429,41.8
311,2019.0,51.0,34.8,0.0,7.442857,18.8,20.0,34.2,26.500000,31.1,23.2,7.657143,37.4


Convert Khatib station

In [127]:
Khatib_df = pd.DataFrame(columns=columns)        
        
index = 0
for i in range(2014, 2020):
    for j in range(1, 54):
        temp_df = raw_Khatib_df[(raw_Khatib_df['Year'] == i) & (raw_Khatib_df['Week'] == j)]
        if len(temp_df) == 0:
            continue
            
        rainSerie = temp_df['Daily Rainfall Total (mm)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        d30RainSerie = temp_df['Highest 30 Min Rainfall (mm)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        d60RainSerie = temp_df['Highest 60 Min Rainfall (mm)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        d120RainSerie = temp_df['Highest 120 Min Rainfall (mm)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        meanTempSerie = temp_df['Mean Temperature (°C)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        maxTempSerie = temp_df['Maximum Temperature (°C)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        minTempSerie = temp_df['Minimum Temperature (°C)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        meanWindSerie = temp_df['Mean Wind Speed (km/h)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        maxWindSerie = temp_df['Max Wind Speed (km/h)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        maxRain = rainSerie.max()
        minRain = rainSerie.min()
        avgRain = rainSerie.sum() / len(rainSerie.dropna())
        max30Rain = d30RainSerie.max()
        max60Rain = d60RainSerie.max()
        max120Rain = d120RainSerie.max()
        avgTemp = meanTempSerie.sum() / len(meanTempSerie.dropna())
        maxTemp = maxTempSerie.max()
        minTemp = minTempSerie.min()
        avgWind = meanWindSerie.sum() / len(meanWindSerie.dropna())
        maxWind = maxWindSerie.max()
        Khatib_df.loc[index] = [i, j, maxRain, minRain, avgRain, max30Rain, max60Rain, max120Rain, avgTemp,
                                         maxTemp, minTemp, avgWind, maxWind]
        index = index + 1
Khatib_df

<ipython-input-127-32d9e88778c9>:30: RuntimeWarning: invalid value encountered in double_scalars
  avgRain = rainSerie.sum() / len(rainSerie.dropna())
<ipython-input-127-32d9e88778c9>:34: RuntimeWarning: invalid value encountered in double_scalars
  avgTemp = meanTempSerie.sum() / len(meanTempSerie.dropna())
<ipython-input-127-32d9e88778c9>:37: RuntimeWarning: invalid value encountered in double_scalars
  avgWind = meanWindSerie.sum() / len(meanWindSerie.dropna())


,Year,Week,Max Daily Rainfall Total (mm),Min Daily Rainfall Total (mm),Average Daily Rainfall Total (mm),Highest 30 Min Rainfall (mm),Highest 60 Min Rainfall (mm),Highest 120 Min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h)
0,2014.0,1.0,19.8,0.0,5.080000,17.2,17.6,19.8,26.760000,32.4,23.7,5.660000,29.2
1,2014.0,2.0,61.8,0.0,12.685714,16.8,24.4,24.6,26.542857,32.7,23.5,4.985714,32.4
2,2014.0,3.0,0.0,0.0,0.000000,0.0,0.0,0.0,26.371429,30.8,23.4,7.771429,34.6
3,2014.0,4.0,0.0,0.0,0.000000,0.0,0.0,0.0,25.857143,30.6,22.4,7.371429,31.7
4,2014.0,5.0,0.0,0.0,0.000000,0.0,0.0,0.0,26.357143,33.0,21.9,7.800000,36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,2019.0,48.0,97.2,0.0,23.000000,57.2,69.8,74.4,27.114286,33.3,23.0,7.100000,41.8
306,2019.0,49.0,45.4,0.0,6.485714,14.6,22.6,40.8,27.100000,32.7,21.8,8.714286,41.8
307,2019.0,50.0,66.4,2.0,28.342857,23.6,27.6,40.4,24.742857,31.6,22.9,6.328571,47.5
308,2019.0,51.0,15.0,0.0,2.714286,14.4,14.6,14.8,25.600000,30.4,22.1,6.785714,33.5


Convert Marina Barrage station

In [128]:
del raw_MarinaBarrage_df['Station']
MarinaBarrage_df = pd.DataFrame(columns=columns)        
        
index = 0
for i in range(2014, 2020):
    for j in range(1, 54):
        temp_df = raw_MarinaBarrage_df[(raw_MarinaBarrage_df['Year'] == i) & (raw_MarinaBarrage_df['Week'] == j)]
        if len(temp_df) == 0:
            continue
            
        rainSerie = temp_df['Daily Rainfall Total (mm)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        d30RainSerie = temp_df['Highest 30 Min Rainfall (mm)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        d60RainSerie = temp_df['Highest 60 Min Rainfall (mm)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        d120RainSerie = temp_df['Highest 120 Min Rainfall (mm)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        meanTempSerie = temp_df['Mean Temperature (°C)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        maxTempSerie = temp_df['Maximum Temperature (°C)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        minTempSerie = temp_df['Minimum Temperature (°C)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        meanWindSerie = temp_df['Mean Wind Speed (km/h)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        maxWindSerie = temp_df['Max Wind Speed (km/h)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        maxRain = rainSerie.max()
        minRain = rainSerie.min()
        avgRain = rainSerie.sum() / len(rainSerie.dropna())
        max30Rain = d30RainSerie.max()
        max60Rain = d60RainSerie.max()
        max120Rain = d120RainSerie.max()
        avgTemp = meanTempSerie.sum() / len(meanTempSerie.dropna())
        maxTemp = maxTempSerie.max()
        minTemp = minTempSerie.min()
        avgWind = meanWindSerie.sum() / len(meanWindSerie.dropna())
        maxWind = maxWindSerie.max()
        MarinaBarrage_df.loc[index] = [i, j, maxRain, minRain, avgRain, max30Rain, max60Rain, max120Rain, avgTemp,
                                         maxTemp, minTemp, avgWind, maxWind]
        index = index + 1
MarinaBarrage_df

<ipython-input-128-142c5d3a006d>:31: RuntimeWarning: invalid value encountered in double_scalars
  avgRain = rainSerie.sum() / len(rainSerie.dropna())
<ipython-input-128-142c5d3a006d>:35: RuntimeWarning: invalid value encountered in double_scalars
  avgTemp = meanTempSerie.sum() / len(meanTempSerie.dropna())
<ipython-input-128-142c5d3a006d>:38: RuntimeWarning: invalid value encountered in double_scalars
  avgWind = meanWindSerie.sum() / len(meanWindSerie.dropna())


,Year,Week,Max Daily Rainfall Total (mm),Min Daily Rainfall Total (mm),Average Daily Rainfall Total (mm),Highest 30 Min Rainfall (mm),Highest 60 Min Rainfall (mm),Highest 120 Min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h)
0,2014.0,1.0,21.8,0.0,6.960000,15.6,17.6,17.8,26.940000,31.9,23.8,8.140000,32.0
1,2014.0,2.0,25.0,0.0,6.942857,19.4,22.0,24.8,27.042857,34.0,24.1,7.442857,44.3
2,2014.0,3.0,1.0,0.0,0.142857,1.0,1.0,1.0,26.500000,31.0,24.4,12.471429,45.0
3,2014.0,4.0,0.0,0.0,0.000000,0.0,0.0,0.0,26.085714,31.0,23.9,12.300000,45.4
4,2014.0,5.0,0.0,0.0,0.000000,0.0,0.0,0.0,26.542857,34.3,23.7,11.842857,37.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,2019.0,48.0,20.0,0.0,4.171429,9.4,10.2,15.0,28.533333,34.5,24.9,6.985714,34.6
309,2019.0,49.0,52.8,0.0,10.560000,36.6,44.4,46.2,27.980000,32.9,23.2,12.340000,45.7
310,2019.0,50.0,54.0,25.6,37.333333,9.8,15.2,20.2,25.000000,29.9,23.7,6.566667,36.7
311,2019.0,51.0,6.6,0.0,1.520000,3.8,3.8,4.4,27.166667,31.1,23.8,7.560000,33.5


Convert Tuas South station

In [143]:
TuasSouth_df = pd.DataFrame(columns=columns)        
        
index = 0
for i in range(2014, 2020):
    for j in range(1, 54):
        temp_df = raw_TuasSouth_df[(raw_MarinaBarrage_df['Year'] == i) & (raw_MarinaBarrage_df['Week'] == j)]
        if len(temp_df) == 0:
            continue
            
        rainSerie = temp_df['Daily Rainfall Total (mm)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        d30RainSerie = temp_df['Highest 30 Min Rainfall (mm)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        d60RainSerie = temp_df['Highest 60 Min Rainfall (mm)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        d120RainSerie = temp_df['Highest 120 Min Rainfall (mm)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        meanTempSerie = temp_df['Mean Temperature (°C)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        maxTempSerie = temp_df['Maximum Temperature (°C)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        minTempSerie = temp_df['Minimum Temperature (°C)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        meanWindSerie = temp_df['Mean Wind Speed (km/h)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        maxWindSerie = temp_df['Max Wind Speed (km/h)'].map(lambda x : float(x) if isfloat(x) else np.nan)
        
        maxRain = rainSerie.max()
        minRain = rainSerie.min()
        avgRain = rainSerie.sum() / len(rainSerie.dropna())
        max30Rain = d30RainSerie.max()
        max60Rain = d60RainSerie.max()
        max120Rain = d120RainSerie.max()
        avgTemp = meanTempSerie.sum() / len(meanTempSerie.dropna())
        maxTemp = maxTempSerie.max()
        minTemp = minTempSerie.min()
        avgWind = meanWindSerie.sum() / len(meanWindSerie.dropna())
        maxWind = maxWindSerie.max()
        TuasSouth_df.loc[index] = [i, j, maxRain, minRain, avgRain, max30Rain, max60Rain, max120Rain, avgTemp,
                                         maxTemp, minTemp, avgWind, maxWind]
        index = index + 1

        
TuasSouth_df

<ipython-input-143-dbe9fb03a71d>:34: RuntimeWarning: invalid value encountered in double_scalars
  avgTemp = meanTempSerie.sum() / len(meanTempSerie.dropna())
<ipython-input-143-dbe9fb03a71d>:30: RuntimeWarning: invalid value encountered in double_scalars
  avgRain = rainSerie.sum() / len(rainSerie.dropna())
<ipython-input-143-dbe9fb03a71d>:37: RuntimeWarning: invalid value encountered in double_scalars
  avgWind = meanWindSerie.sum() / len(meanWindSerie.dropna())


,Year,Week,Max Daily Rainfall Total (mm),Min Daily Rainfall Total (mm),Average Daily Rainfall Total (mm),Highest 30 Min Rainfall (mm),Highest 60 Min Rainfall (mm),Highest 120 Min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h)
0,2014.0,1.0,57.6,0.0,15.040000,30.6,42.2,52.0,26.580000,32.0,23.3,6.340000,72.4
1,2014.0,2.0,21.2,0.0,4.857143,5.0,6.8,9.6,26.642857,31.7,23.2,6.700000,66.2
2,2014.0,3.0,0.0,0.0,0.000000,0.0,0.0,0.0,26.585714,30.9,23.8,8.142857,28.1
3,2014.0,4.0,0.0,0.0,0.000000,0.0,0.0,0.0,26.185714,30.5,23.4,8.300000,27.7
4,2014.0,5.0,0.0,0.0,0.000000,0.0,0.0,0.0,26.742857,32.7,23.1,8.471429,30.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,2019.0,48.0,32.4,0.2,8.428571,12.6,19.4,22.6,27.614286,32.9,23.8,5.657143,40.7
309,2019.0,49.0,15.0,0.0,2.285714,6.8,9.0,11.2,27.771429,32.8,24.0,6.542857,42.8
310,2019.0,50.0,46.6,0.0,16.000000,13.4,22.2,25.0,25.528571,31.3,23.2,6.585714,101.9
311,2019.0,51.0,62.0,0.0,13.771429,32.2,48.2,58.8,27.133333,31.7,24.9,5.714286,28.8


Output weekly weather data of each stations.

In [133]:
AngMoKio_csv_filename = "Datasets/Weather Data/Weekly Ang Mo Kio.csv" 
Changi_csv_filename = "Datasets/Weather Data/Weekly Changi.csv" 
Khatib_csv_filename = "Datasets/Weather Data/Weekly Khatib.csv" 
MarinaBarrage_csv_filename = "Datasets/Weather Data/Weekly Marina Barrage.csv" 
TuasSouth_csv_filename = "Datasets/Weather Data/Weekly Tuas South.csv" 

AngMoKio_df.to_csv(AngMoKio_csv_filename, index=False)
Changi_df.to_csv(Changi_csv_filename, index=False)
Khatib_df.to_csv(Khatib_csv_filename, index=False)
MarinaBarrage_df.to_csv(MarinaBarrage_csv_filename, index=False)
TuasSouth_df.to_csv(TuasSouth_csv_filename, index=False)

Combine stations' data to weather data for whole SG.

In [146]:
Singapore_df = pd.DataFrame(columns=columns)
for i in range(303):
    rowAngMoKio = AngMoKio_df.iloc[i,:]
    rowChangi = Changi_df.iloc[i,:]
    rowKhatib = Khatib_df.iloc[i,:]
    rowMarinaBarrage = MarinaBarrage_df.iloc[i,:]
    rowTuasSouth = TuasSouth_df.iloc[i,:]
    
    maxRainData = [rowAngMoKio['Max Daily Rainfall Total (mm)'], rowChangi['Max Daily Rainfall Total (mm)'],
                   rowKhatib['Max Daily Rainfall Total (mm)'], rowMarinaBarrage['Max Daily Rainfall Total (mm)'],
                   rowTuasSouth['Max Daily Rainfall Total (mm)']]
    minRainData = [rowAngMoKio['Min Daily Rainfall Total (mm)'], rowChangi['Min Daily Rainfall Total (mm)'],
                   rowKhatib['Min Daily Rainfall Total (mm)'], rowMarinaBarrage['Min Daily Rainfall Total (mm)'],
                   rowTuasSouth['Min Daily Rainfall Total (mm)']]
    avgRainData = [rowAngMoKio['Average Daily Rainfall Total (mm)'], rowChangi['Average Daily Rainfall Total (mm)'],
                   rowKhatib['Average Daily Rainfall Total (mm)'], rowMarinaBarrage['Average Daily Rainfall Total (mm)'],
                   rowTuasSouth['Average Daily Rainfall Total (mm)']]
    max30RainData = [rowAngMoKio['Highest 30 Min Rainfall (mm)'], rowChangi['Highest 30 Min Rainfall (mm)'],
                   rowKhatib['Highest 30 Min Rainfall (mm)'], rowMarinaBarrage['Highest 30 Min Rainfall (mm)'],
                   rowTuasSouth['Highest 30 Min Rainfall (mm)']]
    max60RainData = [rowAngMoKio['Highest 60 Min Rainfall (mm)'], rowChangi['Highest 60 Min Rainfall (mm)'],
                   rowKhatib['Highest 60 Min Rainfall (mm)'], rowMarinaBarrage['Highest 60 Min Rainfall (mm)'],
                   rowTuasSouth['Highest 60 Min Rainfall (mm)']]
    max120RainData = [rowAngMoKio['Highest 120 Min Rainfall (mm)'], rowChangi['Highest 120 Min Rainfall (mm)'],
                   rowKhatib['Highest 120 Min Rainfall (mm)'], rowMarinaBarrage['Highest 120 Min Rainfall (mm)'],
                   rowTuasSouth['Highest 120 Min Rainfall (mm)']]
    avgTempData = [rowAngMoKio['Mean Temperature (°C)'], rowChangi['Mean Temperature (°C)'],
                   rowKhatib['Mean Temperature (°C)'], rowMarinaBarrage['Mean Temperature (°C)'],
                   rowTuasSouth['Mean Temperature (°C)']]
    maxTempData = [rowAngMoKio['Maximum Temperature (°C)'], rowChangi['Maximum Temperature (°C)'],
                   rowKhatib['Maximum Temperature (°C)'], rowMarinaBarrage['Maximum Temperature (°C)'],
                   rowTuasSouth['Maximum Temperature (°C)']]
    minTempData = [rowAngMoKio['Minimum Temperature (°C)'], rowChangi['Minimum Temperature (°C)'],
                   rowKhatib['Minimum Temperature (°C)'], rowMarinaBarrage['Minimum Temperature (°C)'],
                   rowTuasSouth['Minimum Temperature (°C)']]
    avgWindData = [rowAngMoKio['Mean Wind Speed (km/h)'], rowChangi['Mean Wind Speed (km/h)'],
                   rowKhatib['Mean Wind Speed (km/h)'], rowMarinaBarrage['Mean Wind Speed (km/h)'],
                   rowTuasSouth['Mean Wind Speed (km/h)']]
    maxWindData = [rowAngMoKio['Max Wind Speed (km/h)'], rowChangi['Max Wind Speed (km/h)'],
                   rowKhatib['Max Wind Speed (km/h)'], rowMarinaBarrage['Max Wind Speed (km/h)'],
                   rowTuasSouth['Max Wind Speed (km/h)']]
    
    maxRainSerie = pd.Series(maxRainData) 
    minRainSerie = pd.Series(minRainData) 
    avgRainSerie = pd.Series(avgRainData) 
    max30RainSerie = pd.Series(max30RainData) 
    max60RainSerie = pd.Series(max60RainData) 
    max120RainSerie = pd.Series(max120RainData) 
    avgTempSerie = pd.Series(avgTempData) 
    maxTempSerie = pd.Series(maxTempData) 
    minTempSerie = pd.Series(minTempData) 
    avgWindSerie = pd.Series(avgWindData) 
    maxWindSerie = pd.Series(maxWindData) 
    
    maxRain = maxRainSerie.max() 
    minRain = minRainSerie.min() 
    avgRain = avgRainSerie.sum() / len(avgRainSerie.dropna()) 
    max30Rain = max30RainSerie.max() 
    max60Rain = max60RainSerie.max() 
    max120Rain = max120RainSerie.max() 
    avgTemp = avgTempSerie.sum() / len(avgTempSerie.dropna())
    maxTemp = maxTempSerie.max()
    minTemp = minTempSerie.min() 
    avgWind = avgWindSerie.sum() / len(avgWindSerie.dropna())
    maxWind = maxWindSerie.max() 
    Singapore_df.loc[i] = [int(rowChangi['Year']), int(rowChangi['Week']), maxRain, minRain, avgRain, max30Rain, max60Rain, max120Rain, avgTemp,
                                         maxTemp, minTemp, avgWind, maxWind]

Singapore_csv_filename = "Datasets/Weather Data/Weekly Singapore.csv" 
Singapore_df.to_csv(Singapore_csv_filename, index=False)

Extract newly increased(last two weeks) on weekly basis

In [37]:
columns = ['date', 'week', 'cases with onset in last 2 weeks']
SG_2014_cases_df = pd.DataFrame(columns=columns) 
raw_2014_location_df = pd.read_csv("Datasets/Dengue Location Cluster Data/2014-Dengue-Cluster.csv")
arr = raw_2014_location_df['date'].unique()
index = 0
for i in arr:
    temp_df = raw_2014_location_df[raw_2014_location_df['date'] == i]
    num = temp_df['cases with onset in laStreet 2 weeks'].sum()
    week = datetime.strptime('20' + str(i), "%Y%m%d").date().isocalendar()[1]
    SG_2014_cases_df.loc[index] = [i, week, num]
    index = index + 1


Singapore_2014_csv_filename = "Datasets/Dengue Location Cluster Data/Singapore-2014-Dengue.csv" 
SG_2014_cases_df.astype('int64').to_csv(Singapore_2014_csv_filename, index=False)


In [44]:
columns = ['date', 'week', 'cases with onset in last 2 weeks']
SG_2015_cases_df = pd.DataFrame(columns=columns) 
raw_2015_location_df = pd.read_csv("Datasets/Dengue Location Cluster Data/2015-Dengue-Cluster.csv")
arr = raw_2015_location_df['date'].unique()
index = 0
for i in arr:
    temp_df = raw_2015_location_df[raw_2015_location_df['date'] == i]
    num = temp_df['cases with onset in laStreet 2 weeks'].astype('int64').sum()
    week = datetime.strptime('20' + str(i), "%Y%m%d").date().isocalendar()[1]
    SG_2015_cases_df.loc[index] = [i, week, num]
    index = index + 1


Singapore_2015_csv_filename = "Datasets/Dengue Location Cluster Data/Singapore-2015-Dengue.csv" 
SG_2015_cases_df.to_csv(Singapore_2015_csv_filename, index=False)



In [47]:
columns = ['date', 'week', 'cases with onset in last 2 weeks']
SG_2016_cases_df = pd.DataFrame(columns=columns) 
raw_2016_location_df = pd.read_csv("Datasets/Dengue Location Cluster Data/2016-Dengue-Cluster.csv")
arr = raw_2016_location_df['date'].unique()
index = 0
for i in arr:
    temp_df = raw_2016_location_df[raw_2016_location_df['date'] == i]
    num = temp_df['cases with onset in laStreet 2 weeks'].sum()
    week = datetime.strptime('20' + str(i), "%Y%m%d").date().isocalendar()[1]
    SG_2016_cases_df.loc[index] = [i, week, num]
    index = index + 1


Singapore_2016_csv_filename = "Datasets/Dengue Location Cluster Data/Singapore-2016-Dengue.csv" 
SG_2016_cases_df.astype('int64').to_csv(Singapore_2016_csv_filename, index=False)



In [51]:
columns = ['date', 'week', 'cases with onset in last 2 weeks']
SG_2017_cases_df = pd.DataFrame(columns=columns) 
raw_2017_location_df = pd.read_csv("Datasets/Dengue Location Cluster Data/raw-2017-Dengue-Cluster.csv")
arr = raw_2017_location_df['S/N'].unique()
index = 0
for i in arr:
    temp_df = raw_2017_location_df[raw_2017_location_df['S/N'] == i]
    num = temp_df['Cases with onset in last 2weeks'].sum()
    week = datetime.strptime('20' + str(i), "%Y%m%d").date().isocalendar()[1]
    SG_2017_cases_df.loc[index] = [i, week, num]
    index = index + 1


Singapore_2017_csv_filename = "Datasets/Dengue Location Cluster Data/Singapore-2017-Dengue.csv" 
SG_2017_cases_df.astype('int64').to_csv(Singapore_2017_csv_filename, index=False)


In [52]:
columns = ['date', 'week', 'cases with onset in last 2 weeks']
SG_2018_cases_df = pd.DataFrame(columns=columns) 
raw_2018_location_df = pd.read_csv("Datasets/Dengue Location Cluster Data/2018-Dengue-Cluster.csv")
arr = raw_2018_location_df['date'].unique()
index = 0
for i in arr:
    temp_df = raw_2018_location_df[raw_2018_location_df['date'] == i]
    num = temp_df['cases with onset in laStreet 2 weeks'].sum()
    week = datetime.strptime('20' + str(i), "%Y%m%d").date().isocalendar()[1]
    SG_2018_cases_df.loc[index] = [i, week, num]
    index = index + 1


Singapore_2018_csv_filename = "Datasets/Dengue Location Cluster Data/Singapore-2018-Dengue.csv" 
SG_2018_cases_df.astype('int64').to_csv(Singapore_2018_csv_filename, index=False)

In [53]:
columns = ['date', 'week', 'cases with onset in last 2 weeks']
SG_2019_cases_df = pd.DataFrame(columns=columns) 
raw_2019_location_df = pd.read_csv("Datasets/Dengue Location Cluster Data/2019-Dengue-Cluster.csv")
arr = raw_2019_location_df['date'].unique()
index = 0
for i in arr:
    temp_df = raw_2019_location_df[raw_2019_location_df['date'] == i]
    num = temp_df['cases with onset in laStreet 2 weeks'].sum()
    week = datetime.strptime('20' + str(i), "%Y%m%d").date().isocalendar()[1]
    SG_2019_cases_df.loc[index] = [i, week, num]
    index = index + 1


Singapore_2019_csv_filename = "Datasets/Dengue Location Cluster Data/Singapore-2019-Dengue.csv" 
SG_2019_cases_df.astype('int64').to_csv(Singapore_2019_csv_filename, index=False)

In [68]:
columns = ['Year', 'Week', 'Max cases with onset in last 2 weeks', 'Average cases with onset in last 2 weeks']
SG_two_weeks_cases = pd.DataFrame(columns=columns)
index = 0
raw_2014 = pd.read_csv("Datasets/Dengue Location Cluster Data/Singapore-2014-Dengue.csv")
raw_2015 = pd.read_csv("Datasets/Dengue Location Cluster Data/Singapore-2015-Dengue.csv") 
raw_2016 = pd.read_csv("Datasets/Dengue Location Cluster Data/Singapore-2016-Dengue.csv") 
raw_2017 = pd.read_csv("Datasets/Dengue Location Cluster Data/Singapore-2017-Dengue.csv") 
raw_2018 = pd.read_csv("Datasets/Dengue Location Cluster Data/Singapore-2018-Dengue.csv") 
raw_2019 = pd.read_csv("Datasets/Dengue Location Cluster Data/Singapore-2019-Dengue.csv") 

arr_2014 = raw_2014['week'].unique()
arr_2015 = raw_2015['week'].unique()
arr_2016 = raw_2016['week'].unique()
arr_2017 = raw_2017['week'].unique()
arr_2018 = raw_2018['week'].unique()
arr_2019 = raw_2019['week'].unique()

for i in arr_2014:
    temp_df = raw_2014[raw_2014['week'] == i]
    temp_sr = temp_df['cases with onset in last 2 weeks']
    n = len(temp_sr.dropna())
    cases_max = temp_sr.max()
    cases_avg = temp_sr.sum() / n
    SG_two_weeks_cases.loc[index] = [2014, i, cases_max, cases_avg]
    index = index + 1

for i in arr_2015:
    temp_df = raw_2015[raw_2015['week'] == i]
    temp_sr = temp_df['cases with onset in last 2 weeks']
    n = len(temp_sr.dropna())
    cases_max = temp_sr.max()
    cases_avg = temp_sr.sum() / n
    SG_two_weeks_cases.loc[index] = [2015, i, cases_max, cases_avg]
    index = index + 1
    
for i in arr_2016:
    temp_df = raw_2016[raw_2016['week'] == i]
    temp_sr = temp_df['cases with onset in last 2 weeks']
    n = len(temp_sr.dropna())
    cases_max = temp_sr.max()
    cases_avg = temp_sr.sum() / n
    SG_two_weeks_cases.loc[index] = [2016, i, cases_max, cases_avg]
    index = index + 1

for i in arr_2017:
    temp_df = raw_2017[raw_2017['week'] == i]
    temp_sr = temp_df['cases with onset in last 2 weeks']
    n = len(temp_sr.dropna())
    cases_max = temp_sr.max()
    cases_avg = temp_sr.sum() / n
    SG_two_weeks_cases.loc[index] = [2017, i, cases_max, cases_avg]
    index = index + 1

for i in arr_2018:
    temp_df = raw_2018[raw_2018['week'] == i]
    temp_sr = temp_df['cases with onset in last 2 weeks']
    n = len(temp_sr.dropna())
    cases_max = temp_sr.max()
    cases_avg = temp_sr.sum() / n
    SG_two_weeks_cases.loc[index] = [2018, i, cases_max, cases_avg]
    index = index + 1
    
for i in arr_2019:
    temp_df = raw_2019[raw_2019['week'] == i]
    temp_sr = temp_df['cases with onset in last 2 weeks']
    n = len(temp_sr.dropna())
    cases_max = temp_sr.max()
    cases_avg = temp_sr.sum() / n
    SG_two_weeks_cases.loc[index] = [2019, i, cases_max, cases_avg]
    index = index + 1
    
SG_two_weeks_cases.to_csv("Datasets/Dengue Location Cluster Data/Singapore-Dengue.csv", index=False)
SG_two_weeks_cases

,Year,Week,Max cases with onset in last 2 weeks,Average cases with onset in last 2 weeks
0,2014.0,1.0,155.0,155.000000
1,2014.0,2.0,224.0,175.500000
2,2014.0,3.0,211.0,202.500000
3,2014.0,4.0,168.0,154.500000
4,2014.0,5.0,130.0,125.333333
...,...,...,...,...
296,2019.0,48.0,349.0,349.000000
297,2019.0,49.0,296.0,296.000000
298,2019.0,50.0,248.0,248.000000
299,2019.0,51.0,215.0,215.000000


In [111]:
columns = ['Year', 'Week', 'Max cases with onset in last 2 weeks', 'Average cases with onset in last 2 weeks']
new_SG_two_weeks_cases = pd.DataFrame(columns=columns)
SG_weather_df = pd.read_csv("Datasets/Weather Data/Weekly Singapore.csv")
columns = ['Year', 'Week', 'Max cases with onset in last 2 weeks', 'Average cases with onset in last 2 weeks', 
               'Max Daily Rainfall Total (mm)', 'Min Daily Rainfall Total (mm)',
               'Average Daily Rainfall Total (mm)', 'Highest 30 Min Rainfall (mm)', 'Highest 60 Min Rainfall (mm)',
                'Highest 120 Min Rainfall (mm)', 'Mean Temperature (°C)', 'Maximum Temperature (°C)',
                'Minimum Temperature (°C)', 'Mean Wind Speed (km/h)', 'Max Wind Speed (km/h)']
index = 0
for i in range(2014, 2020):
    for j in range(1, 54):
        temp_weather_df = SG_weather_df[(SG_weather_df['Year'] == i) & (SG_weather_df['Week'] == j)]
        temp_cases_df = SG_two_weeks_cases[(SG_two_weeks_cases['Year'] == i) & (SG_two_weeks_cases['Week'] == j)]
        hasData = len(temp_cases_df) != 0
        row = []
        if len(temp_weather_df) == 0:
            continue
        if len(temp_cases_df) == 0:
            row = [i, j, np.nan, np.nan]
        else:
            row = temp_cases_df.values.tolist()[0]
        
        new_SG_two_weeks_cases.loc[index] = row
        index = index + 1
del SG_weather_df['Year']  
del SG_weather_df['Week']  
result = pd.concat([new_SG_two_weeks_cases, SG_weather_df], axis=1).reindex(new_SG_two_weeks_cases.index)
result.to_csv("Datasets/Model Training merged data/Singapore Weather and Cases.csv", index=False)

In [173]:
SG_others = pd.read_csv("Datasets/Singapore Population and Cases.csv")
new_SG_two_weeks_cases = pd.read_csv("Datasets/Model Training merged data/Singapore Weather and Cases.csv")
del SG_others['month']  
columns = ['Year','Week','number','population','cases_per_capita','mean_sunshine_hrs','temp_mean_daily_max', 'mean_relative_humidity']
new_SG_others = pd.DataFrame(columns=columns)
index = 0
for i in range(2014, 2020):
    for j in range(1, 54):
        temp_df = new_SG_two_weeks_cases[(new_SG_two_weeks_cases['Year'] == i) & (new_SG_two_weeks_cases['Week'] == j)]
        temp_cases_df = SG_others[(SG_others['year'] == i) & (SG_others['week'] == j)]
        hasData = len(temp_cases_df) != 0
        row = []
        if len(temp_df) == 0:
            continue
        if len(temp_cases_df) == 0:
            row = [i, j, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
        else:
            row = temp_cases_df.values.tolist()[0]
        
        new_SG_others.loc[index] = row
        index = index + 1
del new_SG_others['Year']
del new_SG_others['Week']
new_SG_others.rename(columns={'number':'Cases in week'}, inplace=True)

result = pd.concat([new_SG_two_weeks_cases, new_SG_others], axis=1).reindex(new_SG_two_weeks_cases.index)
result.to_csv("Datasets/Model Training merged data/Singapore Data.csv", index=False)
result


,Year,Week,Max cases with onset in last 2 weeks,Average cases with onset in last 2 weeks,Max Daily Rainfall Total (mm),Min Daily Rainfall Total (mm),Average Daily Rainfall Total (mm),Highest 30 Min Rainfall (mm),Highest 60 Min Rainfall (mm),Highest 120 Min Rainfall (mm),...,Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h),Cases in week,population,cases_per_capita,mean_sunshine_hrs,temp_mean_daily_max,mean_relative_humidity
0,2014.0,1.0,155.0,155.000000,57.6,0.0,6.848000,30.6,42.2,52.0,...,32.4,23.3,7.096000,72.4,436.0,5496512.0,0.000079,5.3,29.6,78.5
1,2014.0,2.0,224.0,175.500000,61.8,0.0,7.931429,19.4,24.4,24.8,...,34.0,23.2,6.460000,66.2,479.0,5496512.0,0.000087,5.3,29.6,78.5
2,2014.0,3.0,211.0,202.500000,1.0,0.0,0.028571,1.0,1.0,1.0,...,31.0,23.3,10.185714,45.0,401.0,5496512.0,0.000073,5.3,29.6,78.5
3,2014.0,4.0,168.0,154.500000,0.0,0.0,0.000000,0.0,0.0,0.0,...,31.0,22.4,9.882857,45.4,336.0,5496512.0,0.000061,5.3,29.6,78.5
4,2014.0,5.0,130.0,125.333333,0.0,0.0,0.000000,0.0,0.0,0.0,...,34.3,21.9,10.137143,42.8,234.0,5496512.0,0.000043,8.8,31.9,74.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,2019.0,38.0,329.0,329.000000,40.5,0.0,2.060000,31.0,40.0,40.5,...,34.0,21.5,10.002857,46.1,299.0,5804337.0,0.000052,6.7,33.0,73.0
299,2019.0,39.0,299.0,299.000000,39.8,0.0,5.743810,14.6,23.6,28.2,...,34.9,22.8,8.284286,47.2,258.0,5804337.0,0.000044,6.5,32.7,78.1
300,2019.0,40.0,218.0,218.000000,53.6,0.0,5.794286,24.8,29.8,30.2,...,34.2,22.5,7.799524,60.5,243.0,5804337.0,0.000042,6.5,32.7,78.1
301,2019.0,41.0,204.0,204.000000,27.0,0.0,4.702857,19.5,23.0,27.0,...,34.5,22.6,7.262857,47.5,226.0,5804337.0,0.000039,6.5,32.7,78.1


In [175]:
national_df = pd.read_csv("Datasets/Model Training merged data/Singapore Data.csv")
psi_max_avg_df = pd.read_csv("Datasets/PSI Data/psi_max_avg.csv")

#columns = ['Year', 'Week', 'Max National', 'Avg National', 'Max South', 'Avg South', 'Max North', 'Avg North', 'Max East'
        #, 'Avg East', 'Max Central', 'Avg Central', 'Max West', 'Avg West']
columns = ['Year', 'Week', 'Max PSI', 'Avg PSI']
psi_max_avg_national_df = pd.DataFrame(columns=columns) 

index = 0

for i in range (2014, 2020):
    psi_year_df = psi_max_avg_df[(psi_max_avg_df['Year'] == i)]
    national_year_df = national_df[national_df['Year'] == i]
    for j in range(1,54):
        psi_week_df = psi_year_df[(psi_year_df['Week'] == j)]
        national_week_df = national_year_df[national_year_df['Week'] == j]
        row = []
        if len(national_week_df) == 0:
            continue
        if len(psi_week_df) == 0:
            row = [i, j, np.nan, np.nan]
        else:
            values = psi_week_df.values.tolist()[0]
            row = [i, j, values[2], values[3]]
            
        psi_max_avg_national_df.loc[index] = row
        index += 1
        
del psi_max_avg_national_df['Year']
del psi_max_avg_national_df['Week']
result = pd.concat([national_df, psi_max_avg_national_df], axis=1).reindex(national_df.index)
result.to_csv("Datasets/Model Training merged data/Singapore Weather PSI and Cases.csv", index=False)

result


,Year,Week,Max cases with onset in last 2 weeks,Average cases with onset in last 2 weeks,Max Daily Rainfall Total (mm),Min Daily Rainfall Total (mm),Average Daily Rainfall Total (mm),Highest 30 Min Rainfall (mm),Highest 60 Min Rainfall (mm),Highest 120 Min Rainfall (mm),...,Mean Wind Speed (km/h),Max Wind Speed (km/h),Cases in week,population,cases_per_capita,mean_sunshine_hrs,temp_mean_daily_max,mean_relative_humidity,Max PSI,Avg PSI
0,2014.0,1.0,155.0,155.000000,57.6,0.0,6.848000,30.6,42.2,52.0,...,7.096000,72.4,436.0,5496512.0,0.000079,5.3,29.6,78.5,NaN,NaN
1,2014.0,2.0,224.0,175.500000,61.8,0.0,7.931429,19.4,24.4,24.8,...,6.460000,66.2,479.0,5496512.0,0.000087,5.3,29.6,78.5,NaN,NaN
2,2014.0,3.0,211.0,202.500000,1.0,0.0,0.028571,1.0,1.0,1.0,...,10.185714,45.0,401.0,5496512.0,0.000073,5.3,29.6,78.5,NaN,NaN
3,2014.0,4.0,168.0,154.500000,0.0,0.0,0.000000,0.0,0.0,0.0,...,9.882857,45.4,336.0,5496512.0,0.000061,5.3,29.6,78.5,NaN,NaN
4,2014.0,5.0,130.0,125.333333,0.0,0.0,0.000000,0.0,0.0,0.0,...,10.137143,42.8,234.0,5496512.0,0.000043,8.8,31.9,74.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,2019.0,38.0,329.0,329.000000,40.5,0.0,2.060000,31.0,40.0,40.5,...,10.002857,46.1,299.0,5804337.0,0.000052,6.7,33.0,73.0,135.0,95.391304
299,2019.0,39.0,299.0,299.000000,39.8,0.0,5.743810,14.6,23.6,28.2,...,8.284286,47.2,258.0,5804337.0,0.000044,6.5,32.7,78.1,118.0,78.018634
300,2019.0,40.0,218.0,218.000000,53.6,0.0,5.794286,24.8,29.8,30.2,...,7.799524,60.5,243.0,5804337.0,0.000042,6.5,32.7,78.1,64.0,56.149068
301,2019.0,41.0,204.0,204.000000,27.0,0.0,4.702857,19.5,23.0,27.0,...,7.262857,47.5,226.0,5804337.0,0.000039,6.5,32.7,78.1,67.0,58.836478
